# Electricity Demand Forecasting: Data Exploration & Feature Analysis

**Objective**: Load, clean, and analyze the electricity demand dataset to prepare for forecasting.

**Workflow**:
1.  **Configuration**: Setup paths and parameters.
2.  **Data Loading**: Load raw data.
3.  **Basic Inspection**: Check data types and missing values.
4.  **EDA**: Visual analysis of demand patterns.


In [ ]:
"""
COMPREHENSIVE DATA EXPLORATION
Electricity Demand Forecasting - ADM Regional Level
Multi-City Weather Features
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings
warnings.filterwarnings('ignore')

# Setup paths
PROJECT_ROOT = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
INPUT_DIR = PROJECT_ROOT / 'data' / 'input'
FIGURES_DIR = PROJECT_ROOT / 'data' / 'output' / 'figures' / 'exploration'
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

# Configuration
CITIES = ['aydin', 'denizli', 'mugla']

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (15, 6)
plt.rcParams['font.size'] = 11

print("="*80)
print("ELECTRICITY DEMAND FORECASTING - COMPREHENSIVE DATA EXPLORATION")
print("ADM Regional Demand with Multi-City Weather Features")
print("="*80)

In [ ]:
# ============================================================================
# SECTION 1: DATA LOADING AND STRUCTURE VERIFICATION
# ============================================================================

print("\n" + "="*80)
print("SECTION 1: DATA LOADING AND STRUCTURE VERIFICATION")
print("="*80)

# Load all city files
dfs = {}
for city in CITIES:
    filepath = INPUT_DIR / f'{city}.csv'
    df = pd.read_csv(filepath)
    df['time'] = pd.to_datetime(df['time'])
    df = df.sort_values('time').reset_index(drop=True)
    dfs[city] = df
    
    print(f"\n{city.upper()}:")
    print(f"  Shape: {df.shape}")
    print(f"  Columns: {df.shape[1]}")
    print(f"  Date range: {df['time'].min()} to {df['time'].max()}")
    print(f"  Duration: {(df['time'].max() - df['time'].min()).days / 365.25:.2f} years")
    print(f"  Total hours: {len(df):,}")

# Verify demand data structure
print("\n" + "-"*80)
print("CRITICAL VERIFICATION: Demand Data Structure")
print("-"*80)

demand_comparison = pd.DataFrame({
    city: dfs[city]['demand'].values for city in CITIES
})

# Statistical comparison
print("\nDemand Statistics by City File:")
print(demand_comparison.describe().T)

# Check if identical
print("\n✓ Checking if demand values are identical across cities:")
all_identical = True
for i, city1 in enumerate(CITIES):
    for city2 in CITIES[i+1:]:
        max_diff = np.abs(dfs[city1]['demand'].values - dfs[city2]['demand'].values).max()
        mean_diff = np.abs(dfs[city1]['demand'].values - dfs[city2]['demand'].values).mean()
        
        if max_diff < 1e-6:  # Effectively zero
            print(f"  {city1.upper()} vs {city2.upper()}: IDENTICAL (max diff: {max_diff:.2e})")
        else:
            print(f"  {city1.upper()} vs {city2.upper()}: DIFFERENT (max diff: {max_diff:.2f}, mean diff: {mean_diff:.2f})")
            all_identical = False

if all_identical:
    print("\n" + "✓"*40)
    print("CONFIRMED: Demand is ADM regional-level (identical across all cities)")
    print("STRATEGY: Use multi-city weather features for regional demand modeling")
    print("✓"*40)
else:
    print("\n" + "⚠"*40)
    print("WARNING: Demand values are NOT identical - investigate data source")
    print("⚠"*40)

# Use first city as base for demand (since identical)
base_df = dfs['aydin'].copy()

In [ ]:
# ============================================================================
# SECTION 2: DATA QUALITY ASSESSMENT
# ============================================================================

print("\n" + "="*80)
print("SECTION 2: DATA QUALITY ASSESSMENT")
print("="*80)

# 2.1 Missing Values
print("\n[2.1] Missing Values Analysis")
print("-"*80)

missing_summary = []
for city in CITIES:
    missing = dfs[city].isnull().sum()
    missing_pct = (missing / len(dfs[city])) * 100
    missing_df = pd.DataFrame({
        'Column': missing.index,
        'Missing': missing.values,
        'Percentage': missing_pct.values
    })
    missing_df = missing_df[missing_df['Missing'] > 0].sort_values('Missing', ascending=False)
    
    print(f"\n{city.upper()}:")
    if len(missing_df) > 0:
        print(missing_df.head(10).to_string(index=False))
    else:
        print("  ✓ No missing values")

# 2.2 Time Continuity
print("\n[2.2] Time Series Continuity")
print("-"*80)

for city in CITIES:
    df = dfs[city]
    time_diff = df['time'].diff()
    expected_diff = pd.Timedelta(hours=1)
    gaps = time_diff[time_diff != expected_diff].dropna()
    
    print(f"\n{city.upper()}:")
    print(f"  Total hours: {len(df):,}")
    print(f"  Expected hours: {int((df['time'].max() - df['time'].min()).total_seconds() / 3600) + 1:,}")
    print(f"  Time gaps found: {len(gaps)}")
    
    if len(gaps) > 0:
        print(f"  Gap details (first 5):")
        gap_indices = gaps.index[:5]
        for idx in gap_indices:
            print(f"    {df.loc[idx-1, 'time']} → {df.loc[idx, 'time']} (gap: {time_diff.loc[idx]})")

# 2.3 Duplicate Timestamps
print("\n[2.3] Duplicate Timestamps")
print("-"*80)

for city in CITIES:
    duplicates = dfs[city]['time'].duplicated().sum()
    print(f"{city.upper()}: {duplicates} duplicates")

# 2.4 Data Types
print("\n[2.4] Data Types Overview")
print("-"*80)
print(base_df.dtypes.value_counts())

In [ ]:
# ============================================================================
# SECTION 3: DEMAND ANALYSIS (REGIONAL)
# ============================================================================

print("\n" + "="*80)
print("SECTION 3: REGIONAL DEMAND ANALYSIS")
print("="*80)

demand = base_df['demand'].values

# 3.1 Basic Statistics
print("\n[3.1] Demand Statistics")
print("-"*80)

demand_stats = base_df['demand'].describe()
print(f"Count:      {demand_stats['count']:,.0f} hours")
print(f"Mean:       {demand_stats['mean']:,.2f} MW")
print(f"Median:     {demand_stats['50%']:,.2f} MW")
print(f"Std Dev:    {demand_stats['std']:,.2f} MW")
print(f"Min:        {demand_stats['min']:,.2f} MW")
print(f"Max:        {demand_stats['max']:,.2f} MW")
print(f"Range:      {demand_stats['max'] - demand_stats['min']:,.2f} MW")
print(f"CV:         {(demand_stats['std'] / demand_stats['mean']):.2%}")
print(f"Skewness:   {stats.skew(demand):.3f}")
print(f"Kurtosis:   {stats.kurtosis(demand):.3f}")

# 3.2 Percentiles
print("\n[3.2] Demand Percentiles")
print("-"*80)
percentiles = [1, 5, 10, 25, 50, 75, 90, 95, 99]
for p in percentiles:
    value = np.percentile(demand, p)
    print(f"  {p:2d}th percentile: {value:,.2f} MW")

# 3.3 Outlier Detection
print("\n[3.3] Outlier Detection")
print("-"*80)

mean_demand = demand_stats['mean']
std_demand = demand_stats['std']

# 3-sigma rule
outliers_3sigma_high = base_df[base_df['demand'] > mean_demand + 3*std_demand]
outliers_3sigma_low = base_df[base_df['demand'] < mean_demand - 3*std_demand]

print(f"3-Sigma Method:")
print(f"  High outliers (>μ+3σ): {len(outliers_3sigma_high)} ({len(outliers_3sigma_high)/len(base_df)*100:.3f}%)")
print(f"  Low outliers (<μ-3σ):  {len(outliers_3sigma_low)} ({len(outliers_3sigma_low)/len(base_df)*100:.3f}%)")

# IQR method
Q1 = demand_stats['25%']
Q3 = demand_stats['75%']
IQR = Q3 - Q1
outliers_iqr_high = base_df[base_df['demand'] > Q3 + 1.5*IQR]
outliers_iqr_low = base_df[base_df['demand'] < Q1 - 1.5*IQR]

print(f"\nIQR Method (1.5×IQR):")
print(f"  High outliers: {len(outliers_iqr_high)} ({len(outliers_iqr_high)/len(base_df)*100:.2f}%)")
print(f"  Low outliers:  {len(outliers_iqr_low)} ({len(outliers_iqr_low)/len(base_df)*100:.2f}%)")

# Show extreme cases
print(f"\n[3.4] Extreme Demand Events")
print("-"*80)
print("\nTop 10 Highest Demand Hours:")
top_demands = base_df.nlargest(10, 'demand')[['time', 'demand', 'hour', 'month', 
                                                'is_weekend', 'is_holiday']]
top_demands_display = top_demands.copy()
top_demands_display['demand'] = top_demands_display['demand'].round(2)
print(top_demands_display.to_string(index=False))

print("\nTop 10 Lowest Demand Hours:")
low_demands = base_df.nsmallest(10, 'demand')[['time', 'demand', 'hour', 'month',
                                                 'is_weekend', 'is_holiday']]
low_demands_display = low_demands.copy()
low_demands_display['demand'] = low_demands_display['demand'].round(2)
print(low_demands_display.to_string(index=False))

# Visualization: Demand Distribution
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Histogram
axes[0, 0].hist(demand, bins=100, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 0].axvline(mean_demand, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_demand:.0f}')
axes[0, 0].axvline(mean_demand + 3*std_demand, color='orange', linestyle='--', linewidth=1.5, label='±3σ')
axes[0, 0].axvline(mean_demand - 3*std_demand, color='orange', linestyle='--', linewidth=1.5)
axes[0, 0].set_xlabel('Demand (MW)', fontsize=12)
axes[0, 0].set_ylabel('Frequency', fontsize=12)
axes[0, 0].set_title('Demand Distribution with Outlier Bounds', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(alpha=0.3)

# Q-Q Plot
stats.probplot(demand, dist="norm", plot=axes[0, 1])
axes[0, 1].set_title('Q-Q Plot (Normal Distribution)', fontsize=14, fontweight='bold')
axes[0, 1].grid(alpha=0.3)

# Box plot by year
base_df['year'] = base_df['time'].dt.year
years = sorted(base_df['year'].unique())
year_data = [base_df[base_df['year'] == year]['demand'].values for year in years]
bp = axes[1, 0].boxplot(year_data, labels=years, patch_artist=True)
for patch in bp['boxes']:
    patch.set_facecolor('lightblue')
axes[1, 0].set_xlabel('Year', fontsize=12)
axes[1, 0].set_ylabel('Demand (MW)', fontsize=12)
axes[1, 0].set_title('Demand Distribution by Year', fontsize=14, fontweight='bold')
axes[1, 0].grid(axis='y', alpha=0.3)

# Time series (full period - downsampled for visibility)
# Show every 24th point (daily instead of hourly)
sample_indices = np.arange(0, len(base_df), 24)
axes[1, 1].plot(base_df['time'].iloc[sample_indices], 
                base_df['demand'].iloc[sample_indices], 
                linewidth=0.5, alpha=0.7, color='darkblue')
axes[1, 1].set_xlabel('Time', fontsize=12)
axes[1, 1].set_ylabel('Demand (MW)', fontsize=12)
axes[1, 1].set_title('Demand Time Series (Daily Average)', fontsize=14, fontweight='bold')
axes[1, 1].grid(alpha=0.3)
plt.setp(axes[1, 1].xaxis.get_majorticklabels(), rotation=45)

plt.tight_layout()
plt.savefig(FIGURES_DIR / '01_demand_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Figure saved: {FIGURES_DIR / '01_demand_distribution.png'}")

In [ ]:
# ============================================================================
# SECTION 4: TEMPORAL PATTERNS
# ============================================================================

print("\n" + "="*80)
print("SECTION 4: TEMPORAL PATTERNS")
print("="*80)

# 4.1 Hourly Patterns
print("\n[4.1] Hourly Patterns")
print("-"*80)

hourly_stats = base_df.groupby('hour')['demand'].agg([
    ('mean', 'mean'),
    ('std', 'std'),
    ('min', 'min'),
    ('max', 'max'),
    ('median', 'median')
])

peak_hour = hourly_stats['mean'].idxmax()
trough_hour = hourly_stats['mean'].idxmin()

print(f"Peak hour:        {peak_hour}:00 ({hourly_stats.loc[peak_hour, 'mean']:.0f} MW)")
print(f"Trough hour:      {trough_hour}:00 ({hourly_stats.loc[trough_hour, 'mean']:.0f} MW)")
print(f"Daily range:      {hourly_stats['mean'].max() - hourly_stats['mean'].min():.0f} MW")
print(f"Peak/trough ratio: {hourly_stats['mean'].max() / hourly_stats['mean'].min():.2f}")
print(f"Avg hourly std:   {hourly_stats['std'].mean():.2f} MW")

# 4.2 Daily Patterns
print("\n[4.2] Daily Patterns (Day of Week)")
print("-"*80)

base_df['day_name'] = base_df['time'].dt.day_name()
daily_stats = base_df.groupby('day_name')['demand'].agg(['mean', 'std']).reindex([
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
])

weekday_avg = base_df[base_df['is_weekend'] == 0]['demand'].mean()
weekend_avg = base_df[base_df['is_weekend'] == 1]['demand'].mean()

print(f"Weekday average:  {weekday_avg:.0f} MW")
print(f"Weekend average:  {weekend_avg:.0f} MW")
print(f"Difference:       {weekday_avg - weekend_avg:.0f} MW ({(weekday_avg - weekend_avg)/weekend_avg*100:+.1f}%)")
print(f"\nDaily Statistics:")
print(daily_stats.round(2))

# 4.3 Monthly Patterns
print("\n[4.3] Monthly Patterns")
print("-"*80)

monthly_stats = base_df.groupby(base_df['time'].dt.month)['demand'].agg(['mean', 'std', 'min', 'max'])
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
monthly_stats.index = month_names

peak_month = monthly_stats['mean'].idxmax()
trough_month = monthly_stats['mean'].idxmin()

print(f"Peak month:   {peak_month} ({monthly_stats.loc[peak_month, 'mean']:.0f} MW)")
print(f"Trough month: {trough_month} ({monthly_stats.loc[trough_month, 'mean']:.0f} MW)")
print(f"Seasonal range: {monthly_stats['mean'].max() - monthly_stats['mean'].min():.0f} MW")
print(f"\nMonthly Statistics:")
print(monthly_stats.round(2))

# 4.4 Seasonal Patterns
print("\n[4.4] Seasonal Patterns")
print("-"*80)

seasonal_stats = base_df.groupby('season')['demand'].agg(['mean', 'std', 'count'])
print(seasonal_stats.round(2))

# Visualization: Temporal Patterns
fig, axes = plt.subplots(3, 3, figsize=(18, 14))

# Row 1: Hourly patterns
# 1.1 Average by hour
axes[0, 0].plot(hourly_stats.index, hourly_stats['mean'], marker='o', 
                linewidth=2, markersize=6, color='darkblue')
axes[0, 0].fill_between(hourly_stats.index, 
                         hourly_stats['mean'] - hourly_stats['std'],
                         hourly_stats['mean'] + hourly_stats['std'],
                         alpha=0.3, color='lightblue')
axes[0, 0].set_xlabel('Hour of Day', fontsize=11)
axes[0, 0].set_ylabel('Demand (MW)', fontsize=11)
axes[0, 0].set_title('Average Demand by Hour (±1 SD)', fontsize=12, fontweight='bold')
axes[0, 0].grid(alpha=0.3)
axes[0, 0].set_xticks(range(0, 24, 2))

# 1.2 Weekday vs Weekend by hour
weekday_hourly = base_df[base_df['is_weekend'] == 0].groupby('hour')['demand'].mean()
weekend_hourly = base_df[base_df['is_weekend'] == 1].groupby('hour')['demand'].mean()
axes[0, 1].plot(weekday_hourly.index, weekday_hourly.values, 
                marker='o', label='Weekday', linewidth=2, markersize=5)
axes[0, 1].plot(weekend_hourly.index, weekend_hourly.values, 
                marker='s', label='Weekend', linewidth=2, markersize=5)
axes[0, 1].set_xlabel('Hour of Day', fontsize=11)
axes[0, 1].set_ylabel('Demand (MW)', fontsize=11)
axes[0, 1].set_title('Hourly Demand: Weekday vs Weekend', fontsize=12, fontweight='bold')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(alpha=0.3)
axes[0, 1].set_xticks(range(0, 24, 2))

# 1.3 Hour variance
axes[0, 2].bar(hourly_stats.index, hourly_stats['std'], 
               color='coral', edgecolor='black', alpha=0.7)
axes[0, 2].set_xlabel('Hour of Day', fontsize=11)
axes[0, 2].set_ylabel('Standard Deviation (MW)', fontsize=11)
axes[0, 2].set_title('Demand Variability by Hour', fontsize=12, fontweight='bold')
axes[0, 2].grid(axis='y', alpha=0.3)
axes[0, 2].set_xticks(range(0, 24, 2))

# Row 2: Daily and Monthly patterns
# 2.1 Day of week
axes[1, 0].bar(range(7), daily_stats['mean'].values, 
               color='steelblue', edgecolor='black', alpha=0.7)
axes[1, 0].errorbar(range(7), daily_stats['mean'].values, 
                    yerr=daily_stats['std'].values, fmt='none', 
                    ecolor='black', capsize=5, capthick=2)
axes[1, 0].set_xlabel('Day of Week', fontsize=11)
axes[1, 0].set_ylabel('Demand (MW)', fontsize=11)
axes[1, 0].set_title('Average Demand by Day of Week (±1 SD)', fontsize=12, fontweight='bold')
axes[1, 0].set_xticks(range(7))
axes[1, 0].set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
axes[1, 0].grid(axis='y', alpha=0.3)

# 2.2 Monthly pattern
axes[1, 1].bar(range(12), monthly_stats['mean'].values, 
               color='forestgreen', edgecolor='black', alpha=0.7)
axes[1, 1].set_xlabel('Month', fontsize=11)
axes[1, 1].set_ylabel('Demand (MW)', fontsize=11)
axes[1, 1].set_title('Average Demand by Month', fontsize=12, fontweight='bold')
axes[1, 1].set_xticks(range(12))
axes[1, 1].set_xticklabels(month_names, rotation=45)
axes[1, 1].grid(axis='y', alpha=0.3)

# 2.3 Monthly pattern across years
for year in sorted(base_df['year'].unique()):
    year_data = base_df[base_df['year'] == year]
    monthly_year = year_data.groupby(year_data['time'].dt.month)['demand'].mean()
    axes[1, 2].plot(monthly_year.index, monthly_year.values, 
                    marker='o', label=str(year), linewidth=2, markersize=4, alpha=0.7)
axes[1, 2].set_xlabel('Month', fontsize=11)
axes[1, 2].set_ylabel('Demand (MW)', fontsize=11)
axes[1, 2].set_title('Monthly Demand Pattern by Year', fontsize=12, fontweight='bold')
axes[1, 2].legend(fontsize=9, ncol=2)
axes[1, 2].grid(alpha=0.3)
axes[1, 2].set_xticks(range(1, 13))

# Row 3: Heatmaps and seasonal
# 3.1 Heatmap: Hour vs Day of Week
pivot_hour_dow = base_df.pivot_table(values='demand', index='hour', 
                                      columns='day_of_week', aggfunc='mean')
im1 = axes[2, 0].imshow(pivot_hour_dow.values, aspect='auto', cmap='YlOrRd', origin='lower')
axes[2, 0].set_xlabel('Day of Week', fontsize=11)
axes[2, 0].set_ylabel('Hour of Day', fontsize=11)
axes[2, 0].set_title('Demand Heatmap: Hour vs Day', fontsize=12, fontweight='bold')
axes[2, 0].set_xticks(range(7))
axes[2, 0].set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
axes[2, 0].set_yticks(range(0, 24, 2))
plt.colorbar(im1, ax=axes[2, 0], label='Demand (MW)')

# 3.2 Heatmap: Hour vs Month
pivot_hour_month = base_df.pivot_table(values='demand', index='hour', 
                                        columns=base_df['time'].dt.month, aggfunc='mean')
im2 = axes[2, 1].imshow(pivot_hour_month.values, aspect='auto', cmap='RdYlGn_r', origin='lower')
axes[2, 1].set_xlabel('Month', fontsize=11)
axes[2, 1].set_ylabel('Hour of Day', fontsize=11)
axes[2, 1].set_title('Demand Heatmap: Hour vs Month', fontsize=12, fontweight='bold')
axes[2, 1].set_xticks(range(12))
axes[2, 1].set_xticklabels(month_names, rotation=45)
axes[2, 1].set_yticks(range(0, 24, 2))
plt.colorbar(im2, ax=axes[2, 1], label='Demand (MW)')

# 3.3 Seasonal box plot
season_order = ['winter', 'spring', 'summer', 'fall']
season_data = [base_df[base_df['season'] == s]['demand'].values 
               for s in season_order if s in base_df['season'].unique()]
bp = axes[2, 2].boxplot(season_data, 
                         labels=[s.capitalize() for s in season_order if s in base_df['season'].unique()],
                         patch_artist=True)
colors = ['lightblue', 'lightgreen', 'coral', 'wheat']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
axes[2, 2].set_ylabel('Demand (MW)', fontsize=11)
axes[2, 2].set_title('Demand Distribution by Season', fontsize=12, fontweight='bold')
axes[2, 2].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / '02_temporal_patterns.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Figure saved: {FIGURES_DIR / '02_temporal_patterns.png'}")

In [ ]:
# ============================================================================
# SECTION 5: MULTI-CITY WEATHER ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("SECTION 5: MULTI-CITY WEATHER COMPARISON")
print("="*80)

# 5.1 Temperature Comparison
print("\n[5.1] Temperature Statistics by City")
print("-"*80)

temp_comparison = pd.DataFrame({
    city: dfs[city]['temperature'].describe() for city in CITIES
}).T

print(temp_comparison.round(2))

# Temperature ranges
print("\n[5.2] Temperature Characteristics")
print("-"*80)
for city in CITIES:
    temp = dfs[city]['temperature']
    print(f"\n{city.upper()}:")
    print(f"  Annual range:    {temp.max() - temp.min():.1f}°C")
    print(f"  Typical range:   {temp.quantile(0.05):.1f}°C to {temp.quantile(0.95):.1f}°C (5th-95th percentile)")
    print(f"  Summer avg (JJA): {dfs[city][dfs[city]['month'].isin([6,7,8])]['temperature'].mean():.1f}°C")
    print(f"  Winter avg (DJF): {dfs[city][dfs[city]['month'].isin([12,1,2])]['temperature'].mean():.1f}°C")

# 5.3 Spatial Temperature Variation
print("\n[5.3] Spatial Temperature Variation")
print("-"*80)

# Calculate hourly temperature differences between cities
temp_diff_coastal_inland = dfs['mugla']['temperature'].values - dfs['denizli']['temperature'].values
temp_diff_aydin_denizli = dfs['aydin']['temperature'].values - dfs['denizli']['temperature'].values

print(f"\nTemperature Gradients:")
print(f"  Muğla - Denizli (Coastal-Inland):")
print(f"    Mean difference:   {temp_diff_coastal_inland.mean():.2f}°C")
print(f"    Std difference:    {temp_diff_coastal_inland.std():.2f}°C")
print(f"    Max difference:    {temp_diff_coastal_inland.max():.2f}°C")
print(f"    Min difference:    {temp_diff_coastal_inland.min():.2f}°C")

print(f"\n  Aydın - Denizli:")
print(f"    Mean difference:   {temp_diff_aydin_denizli.mean():.2f}°C")
print(f"    Std difference:    {temp_diff_aydin_denizli.std():.2f}°C")

# Calculate correlation between city temperatures
print("\n[5.4] Temperature Correlation Between Cities")
print("-"*80)

temp_corr = pd.DataFrame({
    city: dfs[city]['temperature'].values for city in CITIES
}).corr()

print(temp_corr.round(3))

# 5.4 Other Weather Variables
print("\n[5.5] Other Weather Variables - Summary Statistics")
print("-"*80)

weather_vars = ['humidity', 'windspeed', 'precipitation', 'cloudcover']

for var in weather_vars:
    print(f"\n{var.upper()}:")
    var_comparison = pd.DataFrame({
        city: dfs[city][var].describe()[['mean', 'std', 'min', 'max']] 
        for city in CITIES if var in dfs[city].columns
    }).T
    print(var_comparison.round(2))

# Visualization: Multi-City Weather
fig, axes = plt.subplots(3, 3, figsize=(18, 14))

# Row 1: Temperature comparisons
# 1.1 Temperature distributions
for city in CITIES:
    axes[0, 0].hist(dfs[city]['temperature'], bins=50, alpha=0.5, 
                    label=city.capitalize(), edgecolor='black')
axes[0, 0].set_xlabel('Temperature (°C)', fontsize=11)
axes[0, 0].set_ylabel('Frequency', fontsize=11)
axes[0, 0].set_title('Temperature Distribution by City', fontsize=12, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(alpha=0.3)

# 1.2 Temperature box plots
temp_data = [dfs[city]['temperature'].values for city in CITIES]
bp = axes[0, 1].boxplot(temp_data, labels=[c.capitalize() for c in CITIES], 
                         patch_artist=True)
for patch, color in zip(bp['boxes'], ['lightblue', 'lightcoral', 'lightgreen']):
    patch.set_facecolor(color)
axes[0, 1].set_ylabel('Temperature (°C)', fontsize=11)
axes[0, 1].set_title('Temperature Distribution Comparison', fontsize=12, fontweight='bold')
axes[0, 1].grid(axis='y', alpha=0.3)

# 1.3 Temperature correlation heatmap
im1 = axes[0, 2].imshow(temp_corr.values, cmap='coolwarm', vmin=0.9, vmax=1.0)
axes[0, 2].set_xticks(range(len(CITIES)))
axes[0, 2].set_yticks(range(len(CITIES)))
axes[0, 2].set_xticklabels([c.capitalize() for c in CITIES])
axes[0, 2].set_yticklabels([c.capitalize() for c in CITIES])
axes[0, 2].set_title('Temperature Correlation Matrix', fontsize=12, fontweight='bold')
for i in range(len(CITIES)):
    for j in range(len(CITIES)):
        axes[0, 2].text(j, i, f'{temp_corr.values[i, j]:.3f}', 
                        ha='center', va='center', fontsize=10)
plt.colorbar(im1, ax=axes[0, 2])

# Row 2: Monthly temperature patterns by city
for city in CITIES:
    monthly_temp = dfs[city].groupby(dfs[city]['time'].dt.month)['temperature'].mean()
    axes[1, 0].plot(monthly_temp.index, monthly_temp.values, 
                    marker='o', label=city.capitalize(), linewidth=2, markersize=5)
axes[1, 0].set_xlabel('Month', fontsize=11)
axes[1, 0].set_ylabel('Temperature (°C)', fontsize=11)
axes[1, 0].set_title('Average Monthly Temperature by City', fontsize=12, fontweight='bold')
axes[1, 0].legend(fontsize=10)
axes[1, 0].grid(alpha=0.3)
axes[1, 0].set_xticks(range(1, 13))
axes[1, 0].set_xticklabels(month_names, rotation=45)

# 2.2 Temperature gradient time series (sample)
sample_hours = 7*24  # 1 week
sample_df = base_df.head(sample_hours).copy()
sample_df['temp_gradient'] = temp_diff_coastal_inland[:sample_hours]
axes[1, 1].plot(sample_df['time'], sample_df['temp_gradient'], 
                linewidth=1, color='purple')
axes[1, 1].axhline(y=0, color='black', linestyle='--', linewidth=1)
axes[1, 1].set_xlabel('Time', fontsize=11)
axes[1, 1].set_ylabel('Temperature Difference (°C)', fontsize=11)
axes[1, 1].set_title('Coastal-Inland Temperature Gradient (Sample Week)', 
                     fontsize=12, fontweight='bold')
axes[1, 1].grid(alpha=0.3)
plt.setp(axes[1, 1].xaxis.get_majorticklabels(), rotation=45)

# 2.3 Humidity comparison
humidity_data = [dfs[city]['humidity'].values for city in CITIES]
bp2 = axes[1, 2].boxplot(humidity_data, labels=[c.capitalize() for c in CITIES],
                          patch_artist=True)
for patch, color in zip(bp2['boxes'], ['lightblue', 'lightcoral', 'lightgreen']):
    patch.set_facecolor(color)
axes[1, 2].set_ylabel('Humidity (%)', fontsize=11)
axes[1, 2].set_title('Humidity Distribution by City', fontsize=12, fontweight='bold')
axes[1, 2].grid(axis='y', alpha=0.3)

# Row 3: Other weather variables
# 3.1 Precipitation comparison
precip_monthly = pd.DataFrame({
    city: dfs[city].groupby(dfs[city]['time'].dt.month)['precipitation'].sum()
    for city in CITIES
})
for city in CITIES:
    axes[2, 0].plot(precip_monthly.index, precip_monthly[city].values,
                    marker='o', label=city.capitalize(), linewidth=2, markersize=5)
axes[2, 0].set_xlabel('Month', fontsize=11)
axes[2, 0].set_ylabel('Total Precipitation (mm)', fontsize=11)
axes[2, 0].set_title('Total Monthly Precipitation by City', fontsize=12, fontweight='bold')
axes[2, 0].legend(fontsize=10)
axes[2, 0].grid(alpha=0.3)
axes[2, 0].set_xticks(range(1, 13))
axes[2, 0].set_xticklabels(month_names, rotation=45)

# 3.2 Wind speed comparison
wind_data = [dfs[city]['windspeed'].values for city in CITIES]
bp3 = axes[2, 1].boxplot(wind_data, labels=[c.capitalize() for c in CITIES],
                          patch_artist=True)
for patch, color in zip(bp3['boxes'], ['lightblue', 'lightcoral', 'lightgreen']):
    patch.set_facecolor(color)
axes[2, 1].set_ylabel('Wind Speed (m/s)', fontsize=11)
axes[2, 1].set_title('Wind Speed Distribution by City', fontsize=12, fontweight='bold')
axes[2, 1].grid(axis='y', alpha=0.3)

# 3.3 Cloud cover comparison
cloud_data = [dfs[city]['cloudcover'].values for city in CITIES]
bp4 = axes[2, 2].boxplot(cloud_data, labels=[c.capitalize() for c in CITIES],
                          patch_artist=True)
for patch, color in zip(bp4['boxes'], ['lightblue', 'lightcoral', 'lightgreen']):
    patch.set_facecolor(color)
axes[2, 2].set_ylabel('Cloud Cover (%)', fontsize=11)
axes[2, 2].set_title('Cloud Cover Distribution by City', fontsize=12, fontweight='bold')
axes[2, 2].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / '03_multi_city_weather.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Figure saved: {FIGURES_DIR / '03_multi_city_weather.png'}")

In [ ]:
# ============================================================================
# SECTION 6: DEMAND-WEATHER RELATIONSHIPS
# ============================================================================

print("\n" + "="*80)
print("SECTION 6: DEMAND-WEATHER RELATIONSHIPS")
print("="*80)

# 6.1 Correlation Analysis - Each City's Weather vs Regional Demand
print("\n[6.1] Correlation: City-Specific Weather vs Regional Demand")
print("-"*80)

weather_vars_analysis = ['temperature', 'humidity', 'dewpoint', 'windspeed', 
                         'precipitation', 'cloudcover', 'pressure_msl']

correlation_results = {}

for city in CITIES:
    print(f"\n{city.upper()}:")
    correlations = {}
    for var in weather_vars_analysis:
        if var in dfs[city].columns:
            corr = np.corrcoef(dfs[city][var].values, dfs[city]['demand'].values)[0, 1]
            correlations[var] = corr
            print(f"  {var:20s}: {corr:+.3f}")
    correlation_results[city] = correlations

# 6.2 Temperature-Demand Relationship (Most Important)
print("\n[6.2] Temperature-Demand Relationship Analysis")
print("-"*80)

for city in CITIES:
    temp = dfs[city]['temperature'].values
    demand = dfs[city]['demand'].values
    
    # Linear correlation
    corr_linear = np.corrcoef(temp, demand)[0, 1]
    
    # Polynomial fit to check non-linearity
    z = np.polyfit(temp, demand, 2)
    p = np.poly1d(z)
    r2_poly = 1 - (np.sum((demand - p(temp))**2) / np.sum((demand - demand.mean())**2))
    
    print(f"\n{city.upper()}:")
    print(f"  Linear correlation:      {corr_linear:+.3f}")
    print(f"  Polynomial R²:           {r2_poly:.3f}")
    
    # Find optimal temperature (minimum demand)
    temp_range = np.linspace(temp.min(), temp.max(), 1000)
    optimal_temp = temp_range[np.argmin(p(temp_range))]
    print(f"  Optimal temperature:     {optimal_temp:.1f}°C (minimum demand)")

# 6.3 Temperature Bins Analysis
print("\n[6.3] Demand by Temperature Bins")
print("-"*80)

# Create temperature bins
temp_bins = np.arange(-5, 45, 5)
bin_labels = [f"{temp_bins[i]}-{temp_bins[i+1]}°C" for i in range(len(temp_bins)-1)]

for city in CITIES:
    dfs[city]['temp_bin'] = pd.cut(dfs[city]['temperature'], bins=temp_bins, labels=bin_labels)
    bin_stats = dfs[city].groupby('temp_bin')['demand'].agg(['mean', 'count'])
    
    print(f"\n{city.upper()}:")
    print(bin_stats.round(2))

# 6.4 Heating vs Cooling Demand Proxies
print("\n[6.4] Heating and Cooling Demand Analysis")
print("-"*80)

# Calculate heating and cooling degree hours for each city
heating_base = 18  # Base temperature for heating
cooling_base = 24  # Base temperature for cooling

for city in CITIES:
    temp = dfs[city]['temperature']
    demand = dfs[city]['demand']
    
    # Calculate degree hours
    heating_dh = np.maximum(heating_base - temp, 0)
    cooling_dh = np.maximum(temp - cooling_base, 0)
    
    # Correlation with demand
    corr_heating = np.corrcoef(heating_dh, demand)[0, 1]
    corr_cooling = np.corrcoef(cooling_dh, demand)[0, 1]
    
    print(f"\n{city.upper()}:")
    print(f"  Heating degree hours correlation:  {corr_heating:+.3f}")
    print(f"  Cooling degree hours correlation:  {corr_cooling:+.3f}")
    
    # Average demand in different temperature regimes
    cold_mask = temp < heating_base
    comfortable_mask = (temp >= heating_base) & (temp <= cooling_base)
    hot_mask = temp > cooling_base
    
    print(f"  Avg demand when cold (<{heating_base}°C):       {demand[cold_mask].mean():.0f} MW")
    print(f"  Avg demand when comfortable ({heating_base}-{cooling_base}°C): {demand[comfortable_mask].mean():.0f} MW")
    print(f"  Avg demand when hot (>{cooling_base}°C):        {demand[hot_mask].mean():.0f} MW")

# 6.5 Multi-City Weather Impact
print("\n[6.5] Spatial Weather Heterogeneity Impact")
print("-"*80)

# Calculate regional temperature statistics
regional_temp_mean = np.mean([dfs[city]['temperature'].values for city in CITIES], axis=0)
regional_temp_std = np.std([dfs[city]['temperature'].values for city in CITIES], axis=0)
regional_temp_range = (
    np.max([dfs[city]['temperature'].values for city in CITIES], axis=0) -
    np.min([dfs[city]['temperature'].values for city in CITIES], axis=0)
)

demand = base_df['demand'].values

# Correlations
corr_mean = np.corrcoef(regional_temp_mean, demand)[0, 1]
corr_std = np.corrcoef(regional_temp_std, demand)[0, 1]
corr_range = np.corrcoef(regional_temp_range, demand)[0, 1]

print(f"Regional Temperature Metrics vs Demand:")
print(f"  Mean temperature:         {corr_mean:+.3f}")
print(f"  Std dev (heterogeneity):  {corr_std:+.3f}")
print(f"  Range (spatial variation): {corr_range:+.3f}")

print(f"\nSpatial Temperature Variation Statistics:")
print(f"  Mean std across cities:    {regional_temp_std.mean():.2f}°C")
print(f"  Max std across cities:     {regional_temp_std.max():.2f}°C")
print(f"  Mean range across cities:  {regional_temp_range.mean():.2f}°C")
print(f"  Max range across cities:   {regional_temp_range.max():.2f}°C")

# Visualization: Demand-Weather Relationships
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Row 1: Temperature-Demand scatter plots for each city
for idx, city in enumerate(CITIES):
    ax = fig.add_subplot(gs[0, idx])
    
    # Sample data for visibility (every 10th point)
    sample_indices = np.arange(0, len(dfs[city]), 10)
    temp_sample = dfs[city]['temperature'].iloc[sample_indices]
    demand_sample = dfs[city]['demand'].iloc[sample_indices]
    
    # Scatter plot
    ax.scatter(temp_sample, demand_sample, alpha=0.3, s=2, c='blue')
    
    # Polynomial fit
    z = np.polyfit(dfs[city]['temperature'], dfs[city]['demand'], 2)
    p = np.poly1d(z)
    temp_range = np.linspace(dfs[city]['temperature'].min(), 
                             dfs[city]['temperature'].max(), 100)
    ax.plot(temp_range, p(temp_range), 'r-', linewidth=2, label='Polynomial fit')
    
    ax.set_xlabel('Temperature (°C)', fontsize=11)
    ax.set_ylabel('Demand (MW)', fontsize=11)
    ax.set_title(f'{city.capitalize()} - Temp vs Demand', fontsize=12, fontweight='bold')
    ax.grid(alpha=0.3)
    ax.legend(fontsize=9)

# Row 2: Demand by temperature bins
ax1 = fig.add_subplot(gs[1, 0])
for city in CITIES:
    bin_means = dfs[city].groupby('temp_bin')['demand'].mean()
    ax1.plot(range(len(bin_means)), bin_means.values, 
             marker='o', label=city.capitalize(), linewidth=2, markersize=6)
ax1.set_xlabel('Temperature Bin', fontsize=11)
ax1.set_ylabel('Average Demand (MW)', fontsize=11)
ax1.set_title('Average Demand by Temperature Range', fontsize=12, fontweight='bold')
ax1.set_xticks(range(len(bin_labels)))
ax1.set_xticklabels(bin_labels, rotation=45, ha='right')
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3)

# Row 2: Heating vs Cooling demand
ax2 = fig.add_subplot(gs[1, 1])
city_data = dfs['aydin']  # Use one city as example
temp = city_data['temperature']
demand = city_data['demand']

# Calculate degree hours
heating_dh = np.maximum(18 - temp, 0)
cooling_dh = np.maximum(temp - 24, 0)

# Sample for visibility
sample_indices = np.arange(0, len(city_data), 10)
ax2.scatter(heating_dh.iloc[sample_indices], demand.iloc[sample_indices], 
           alpha=0.4, s=10, c='blue', label='Heating')
ax2.scatter(cooling_dh.iloc[sample_indices], demand.iloc[sample_indices], 
           alpha=0.4, s=10, c='red', label='Cooling')
ax2.set_xlabel('Degree Hours', fontsize=11)
ax2.set_ylabel('Demand (MW)', fontsize=11)
ax2.set_title('Heating/Cooling Degree Hours vs Demand', fontsize=12, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3)

# Row 2: Correlation heatmap
ax3 = fig.add_subplot(gs[1, 2])
corr_matrix = []
corr_labels = []
for city in CITIES:
    for var in ['temperature', 'humidity', 'windspeed']:
        if var in dfs[city].columns:
            corr = np.corrcoef(dfs[city][var].values, dfs[city]['demand'].values)[0, 1]
            corr_matrix.append(corr)
            corr_labels.append(f"{city.capitalize()}\n{var}")

corr_array = np.array(corr_matrix).reshape(len(CITIES), 3)
im = ax3.imshow(corr_array, cmap='RdYlGn', vmin=-1, vmax=1, aspect='auto')
ax3.set_xticks(range(3))
ax3.set_xticklabels(['Temperature', 'Humidity', 'Wind Speed'])
ax3.set_yticks(range(len(CITIES)))
ax3.set_yticklabels([c.capitalize() for c in CITIES])
ax3.set_title('Weather-Demand Correlations', fontsize=12, fontweight='bold')
for i in range(len(CITIES)):
    for j in range(3):
        ax3.text(j, i, f'{corr_array[i, j]:+.2f}', 
                ha='center', va='center', fontsize=10)
plt.colorbar(im, ax=ax3, label='Correlation')

# Row 3: Regional temperature metrics
ax4 = fig.add_subplot(gs[2, 0])
sample_hours = 30*24  # 30 days
sample_indices = slice(0, sample_hours)
ax4.plot(base_df['time'].iloc[sample_indices], 
         regional_temp_mean[sample_indices], 
         label='Mean', linewidth=2)
ax4.fill_between(base_df['time'].iloc[sample_indices],
                 regional_temp_mean[sample_indices] - regional_temp_std[sample_indices],
                 regional_temp_mean[sample_indices] + regional_temp_std[sample_indices],
                 alpha=0.3, label='±1 SD (heterogeneity)')
ax4.set_xlabel('Time', fontsize=11)
ax4.set_ylabel('Temperature (°C)', fontsize=11)
ax4.set_title('Regional Temperature with Spatial Variation (30 days)', 
             fontsize=12, fontweight='bold')
ax4.legend(fontsize=10)
ax4.grid(alpha=0.3)
plt.setp(ax4.xaxis.get_majorticklabels(), rotation=45)

# Row 3: Spatial heterogeneity vs demand
ax5 = fig.add_subplot(gs[2, 1])
sample_indices = np.arange(0, len(base_df), 10)
ax5.scatter(regional_temp_std[sample_indices], 
           demand[sample_indices],
           alpha=0.3, s=10, c='purple')
ax5.set_xlabel('Temperature Std Dev Across Cities (°C)', fontsize=11)
ax5.set_ylabel('Demand (MW)', fontsize=11)
ax5.set_title('Spatial Temperature Heterogeneity vs Demand', 
             fontsize=12, fontweight='bold')
ax5.grid(alpha=0.3)

# Row 3: Hour-Temperature interaction on demand
ax6 = fig.add_subplot(gs[2, 2])
# Create bins for temperature and hour
temp_hour_pivot = base_df.copy()
temp_hour_pivot['temp_category'] = pd.cut(temp_hour_pivot['temperature'], 
                                           bins=5, labels=['Very Cold', 'Cold', 'Moderate', 'Warm', 'Hot'])
hourly_by_temp = temp_hour_pivot.groupby(['hour', 'temp_category'])['demand'].mean().unstack()

for col in hourly_by_temp.columns:
    ax6.plot(hourly_by_temp.index, hourly_by_temp[col], 
            marker='o', label=col, linewidth=2, markersize=4, alpha=0.7)
ax6.set_xlabel('Hour of Day', fontsize=11)
ax6.set_ylabel('Average Demand (MW)', fontsize=11)
ax6.set_title('Hourly Demand Pattern by Temperature Category', 
             fontsize=12, fontweight='bold')
ax6.legend(fontsize=9, ncol=2)
ax6.grid(alpha=0.3)
ax6.set_xticks(range(0, 24, 2))

plt.savefig(FIGURES_DIR / '04_demand_weather_relationships.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Figure saved: {FIGURES_DIR / '04_demand_weather_relationships.png'}")

In [ ]:
# ============================================================================
# SECTION 7: AUTOCORRELATION AND STATIONARITY ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("SECTION 7: AUTOCORRELATION AND STATIONARITY ANALYSIS")
print("="*80)

# 7.1 Autocorrelation Analysis
print("\n[7.1] Autocorrelation Analysis")
print("-"*80)

from statsmodels.tsa.stattools import acf, pacf

demand_series = base_df['demand'].values

# Calculate ACF and PACF
acf_values = acf(demand_series, nlags=168)  # 1 week
pacf_values = pacf(demand_series, nlags=168)

# Find significant lags
significant_lags = []
confidence_interval = 1.96 / np.sqrt(len(demand_series))

for lag in range(1, 169):
    if abs(acf_values[lag]) > confidence_interval:
        significant_lags.append(lag)

print(f"Number of significant lags (up to 168h): {len(significant_lags)}")
print(f"\nKey autocorrelation values:")
key_lags = [1, 24, 48, 72, 168, 336]
for lag in key_lags:
    if lag < len(acf_values):
        print(f"  Lag {lag:3d}h: {acf_values[lag]:.3f}")

# 7.2 Stationarity Tests
print("\n[7.2] Stationarity Tests")
print("-"*80)

from statsmodels.tsa.stattools import adfuller, kpss

# Augmented Dickey-Fuller test
adf_result = adfuller(demand_series, autolag='AIC')
print(f"\nAugmented Dickey-Fuller Test:")
print(f"  ADF Statistic:   {adf_result[0]:.4f}")
print(f"  p-value:         {adf_result[1]:.4f}")
print(f"  Critical values:")
for key, value in adf_result[4].items():
    print(f"    {key}: {value:.4f}")

if adf_result[1] < 0.05:
    print(f"  → Reject null hypothesis: Series is STATIONARY")
else:
    print(f"  → Fail to reject null hypothesis: Series is NON-STATIONARY")

# KPSS test
kpss_result = kpss(demand_series, regression='ct', nlags='auto')
print(f"\nKPSS Test:")
print(f"  KPSS Statistic:  {kpss_result[0]:.4f}")
print(f"  p-value:         {kpss_result[1]:.4f}")
print(f"  Critical values:")
for key, value in kpss_result[3].items():
    print(f"    {key}: {value:.4f}")

if kpss_result[1] > 0.05:
    print(f"  → Fail to reject null hypothesis: Series is STATIONARY")
else:
    print(f"  → Reject null hypothesis: Series is NON-STATIONARY")

# 7.3 Differencing Analysis
print("\n[7.3] Impact of Differencing")
print("-"*80)

# First difference
demand_diff1 = np.diff(demand_series)
adf_diff1 = adfuller(demand_diff1, autolag='AIC')

# Seasonal difference (24h)
demand_diff24 = demand_series[24:] - demand_series[:-24]
adf_diff24 = adfuller(demand_diff24, autolag='AIC')

print(f"Original series:")
print(f"  ADF statistic: {adf_result[0]:.4f}, p-value: {adf_result[1]:.4f}")

print(f"\nFirst difference:")
print(f"  ADF statistic: {adf_diff1[0]:.4f}, p-value: {adf_diff1[1]:.4f}")
if adf_diff1[1] < 0.05:
    print(f"  → STATIONARY after first difference")

print(f"\nSeasonal difference (24h):")
print(f"  ADF statistic: {adf_diff24[0]:.4f}, p-value: {adf_diff24[1]:.4f}")
if adf_diff24[1] < 0.05:
    print(f"  → STATIONARY after seasonal difference")

# 7.4 Decomposition
print("\n[7.4] Time Series Decomposition")
print("-"*80)

# Use a subset for decomposition (computational efficiency)
decomp_period = 24 * 365  # 1 year
decomp_data = base_df.iloc[:decomp_period].copy()
decomp_data = decomp_data.set_index('time')

# Additive decomposition
decomposition = seasonal_decompose(decomp_data['demand'], 
                                   model='additive', 
                                   period=24*7)  # Weekly seasonality

trend_strength = 1 - (np.var(decomposition.resid.dropna()) / 
                      np.var(decomposition.trend.dropna() + decomposition.resid.dropna()))
seasonal_strength = 1 - (np.var(decomposition.resid.dropna()) / 
                         np.var(decomposition.seasonal.dropna() + decomposition.resid.dropna()))

print(f"Decomposition Statistics (1 year sample):")
print(f"  Trend strength:     {trend_strength:.3f}")
print(f"  Seasonal strength:  {seasonal_strength:.3f}")
print(f"  Residual std:       {decomposition.resid.std():.2f} MW")

# Visualization: Autocorrelation and Stationarity
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Row 1: ACF and PACF
ax1 = fig.add_subplot(gs[0, :2])
plot_acf(demand_series, lags=168, ax=ax1, alpha=0.05)
ax1.set_xlabel('Lag (hours)', fontsize=11)
ax1.set_ylabel('Autocorrelation', fontsize=11)
ax1.set_title('Autocorrelation Function (ACF) - 168 hours', 
             fontsize=12, fontweight='bold')
ax1.grid(alpha=0.3)

ax2 = fig.add_subplot(gs[0, 2])
plot_pacf(demand_series, lags=48, ax=ax2, alpha=0.05)
ax2.set_xlabel('Lag (hours)', fontsize=11)
ax2.set_ylabel('Partial Autocorrelation', fontsize=11)
ax2.set_title('Partial ACF - 48 hours', fontsize=12, fontweight='bold')
ax2.grid(alpha=0.3)

# Row 2: Decomposition
ax3 = fig.add_subplot(gs[1, 0])
ax3.plot(decomposition.observed.index, decomposition.observed.values, linewidth=0.5)
ax3.set_ylabel('Observed', fontsize=10)
ax3.set_title('Original Series', fontsize=11, fontweight='bold')
ax3.grid(alpha=0.3)
plt.setp(ax3.xaxis.get_majorticklabels(), rotation=45)

ax4 = fig.add_subplot(gs[1, 1])
ax4.plot(decomposition.trend.index, decomposition.trend.values, linewidth=1, color='orange')
ax4.set_ylabel('Trend', fontsize=10)
ax4.set_title('Trend Component', fontsize=11, fontweight='bold')
ax4.grid(alpha=0.3)
plt.setp(ax4.xaxis.get_majorticklabels(), rotation=45)

ax5 = fig.add_subplot(gs[1, 2])
# Show one week of seasonal component
seasonal_week = decomposition.seasonal[:24*7]
ax5.plot(range(len(seasonal_week)), seasonal_week.values, linewidth=2, color='green')
ax5.set_xlabel('Hour of Week', fontsize=10)
ax5.set_ylabel('Seasonal', fontsize=10)
ax5.set_title('Seasonal Component (1 week)', fontsize=11, fontweight='bold')
ax5.grid(alpha=0.3)

# Row 3: Differencing analysis
ax6 = fig.add_subplot(gs[2, 0])
sample_hours = 7*24  # 1 week sample
ax6.plot(range(sample_hours), demand_series[:sample_hours], linewidth=1)
ax6.set_xlabel('Hour', fontsize=10)
ax6.set_ylabel('Demand (MW)', fontsize=10)
ax6.set_title('Original Series (1 week sample)', fontsize=11, fontweight='bold')
ax6.grid(alpha=0.3)

ax7 = fig.add_subplot(gs[2, 1])
ax7.plot(range(sample_hours-1), demand_diff1[:sample_hours-1], linewidth=1, color='red')
ax7.axhline(y=0, color='black', linestyle='--', linewidth=1)
ax7.set_xlabel('Hour', fontsize=10)
ax7.set_ylabel('Difference', fontsize=10)
ax7.set_title('First Difference', fontsize=11, fontweight='bold')
ax7.grid(alpha=0.3)

ax8 = fig.add_subplot(gs[2, 2])
ax8.plot(range(sample_hours-24), demand_diff24[:sample_hours-24], linewidth=1, color='purple')
ax8.axhline(y=0, color='black', linestyle='--', linewidth=1)
ax8.set_xlabel('Hour', fontsize=10)
ax8.set_ylabel('Difference', fontsize=10)
ax8.set_title('Seasonal Difference (24h)', fontsize=11, fontweight='bold')
ax8.grid(alpha=0.3)

plt.savefig(FIGURES_DIR / '05_autocorrelation_stationarity.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Figure saved: {FIGURES_DIR / '05_autocorrelation_stationarity.png'}")

In [ ]:
# ============================================================================
# SECTION 8: FEATURE CORRELATION ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("SECTION 8: FEATURE CORRELATION ANALYSIS")
print("="*80)

# 8.1 Prepare feature correlation matrix
print("\n[8.1] Computing Feature Correlations")
print("-"*80)

# Select relevant features for correlation analysis
feature_cols = ['demand', 'hour', 'day_of_week', 'month', 'is_weekend', 
                'is_holiday', 'temperature', 'humidity', 'dewpoint', 
                'windspeed', 'cloudcover', 'precipitation', 'pressure_msl']

# Available features
available_features = [col for col in feature_cols if col in base_df.columns]

# Create correlation matrix
corr_matrix = base_df[available_features].corr()

# Extract demand correlations
demand_corr = corr_matrix['demand'].sort_values(ascending=False)

print(f"\nTop correlations with demand:")
print(demand_corr.head(10).to_string())

print(f"\nBottom correlations with demand:")
print(demand_corr.tail(5).to_string())

# 8.2 Multi-city weather correlations with demand
print("\n[8.2] Multi-City Weather Feature Correlations")
print("-"*80)

multicity_corr = {}
for city in CITIES:
    city_weather_cols = ['temperature', 'humidity', 'dewpoint', 'windspeed']
    city_corr = {}
    for col in city_weather_cols:
        if col in dfs[city].columns:
            corr = np.corrcoef(dfs[city][col].values, dfs[city]['demand'].values)[0, 1]
            city_corr[col] = corr
    multicity_corr[city] = city_corr

multicity_corr_df = pd.DataFrame(multicity_corr).T
print(multicity_corr_df.round(3))

# 8.3 Feature multicollinearity
print("\n[8.3] Multicollinearity Analysis (VIF)")
print("-"*80)

from statsmodels.stats.outliers_influence import variance_inflation_factor

# Select numerical features (exclude demand)
numerical_features = [col for col in available_features 
                     if col != 'demand' and base_df[col].dtype in ['int64', 'float64']]

# Calculate VIF
vif_data = pd.DataFrame()
vif_data["Feature"] = numerical_features
vif_data["VIF"] = [variance_inflation_factor(base_df[numerical_features].values, i) 
                   for i in range(len(numerical_features))]
vif_data = vif_data.sort_values('VIF', ascending=False)

print(vif_data.to_string(index=False))
print(f"\nInterpretation:")
print(f"  VIF < 5:   Low multicollinearity")
print(f"  VIF 5-10:  Moderate multicollinearity")
print(f"  VIF > 10:  High multicollinearity (consider removing)")

high_vif = vif_data[vif_data['VIF'] > 10]
if len(high_vif) > 0:
    print(f"\n⚠ Features with high VIF:")
    print(high_vif.to_string(index=False))

# Visualization: Feature Correlations
fig = plt.figure(figsize=(18, 10))
gs = fig.add_gridspec(2, 3, hspace=0.3, wspace=0.3)

# Full correlation matrix heatmap
ax1 = fig.add_subplot(gs[0, :2])
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', 
            cmap='RdYlGn', center=0, vmin=-1, vmax=1,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8},
            ax=ax1, annot_kws={'size': 8})
ax1.set_title('Feature Correlation Matrix', fontsize=12, fontweight='bold')

# Demand correlations bar plot
ax2 = fig.add_subplot(gs[0, 2])
demand_corr_plot = demand_corr.drop('demand').head(10)
colors = ['green' if x > 0 else 'red' for x in demand_corr_plot.values]
ax2.barh(range(len(demand_corr_plot)), demand_corr_plot.values, color=colors, alpha=0.7)
ax2.set_yticks(range(len(demand_corr_plot)))
ax2.set_yticklabels(demand_corr_plot.index, fontsize=10)
ax2.set_xlabel('Correlation with Demand', fontsize=11)
ax2.set_title('Top 10 Demand Correlations', fontsize=12, fontweight='bold')
ax2.axvline(x=0, color='black', linestyle='-', linewidth=1)
ax2.grid(axis='x', alpha=0.3)

# Multi-city weather correlation comparison
ax3 = fig.add_subplot(gs[1, 0])
x = np.arange(len(multicity_corr_df.columns))
width = 0.25
for i, city in enumerate(CITIES):
    ax3.bar(x + i*width, multicity_corr_df.loc[city].values, 
           width, label=city.capitalize(), alpha=0.7)
ax3.set_xlabel('Weather Variable', fontsize=11)
ax3.set_ylabel('Correlation with Demand', fontsize=11)
ax3.set_title('Weather Correlations by City', fontsize=12, fontweight='bold')
ax3.set_xticks(x + width)
ax3.set_xticklabels(multicity_corr_df.columns, rotation=45, ha='right')
ax3.legend(fontsize=10)
ax3.grid(axis='y', alpha=0.3)
ax3.axhline(y=0, color='black', linestyle='-', linewidth=1)

# VIF plot
ax4 = fig.add_subplot(gs[1, 1])
colors_vif = ['red' if x > 10 else 'orange' if x > 5 else 'green' 
              for x in vif_data['VIF'].values]
ax4.barh(range(len(vif_data)), vif_data['VIF'].values, color=colors_vif, alpha=0.7)
ax4.set_yticks(range(len(vif_data)))
ax4.set_yticklabels(vif_data['Feature'].values, fontsize=9)
ax4.set_xlabel('VIF', fontsize=11)
ax4.set_title('Variance Inflation Factor', fontsize=12, fontweight='bold')
ax4.axvline(x=5, color='orange', linestyle='--', linewidth=1, label='VIF=5')
ax4.axvline(x=10, color='red', linestyle='--', linewidth=1, label='VIF=10')
ax4.legend(fontsize=9)
ax4.grid(axis='x', alpha=0.3)

# Scatter matrix for key features
ax5 = fig.add_subplot(gs[1, 2])
key_features = ['demand', 'temperature', 'hour']
scatter_df = base_df[key_features].sample(n=1000, random_state=42)  # Sample for visibility
pd.plotting.scatter_matrix(scatter_df, ax=ax5, alpha=0.3, figsize=(6, 6), 
                          diagonal='hist', s=10)
ax5.set_title('Scatter Matrix (Key Features)', fontsize=12, fontweight='bold')

plt.savefig(FIGURES_DIR / '06_feature_correlations.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Figure saved: {FIGURES_DIR / '06_feature_correlations.png'}")

In [ ]:
# ============================================================================
# SECTION 9: SUMMARY AND RECOMMENDATIONS
# ============================================================================

print("\n" + "="*80)
print("SECTION 9: DATA EXPLORATION SUMMARY AND RECOMMENDATIONS")
print("="*80)

print("\n" + "="*80)
print("KEY FINDINGS")
print("="*80)

print("\n[1] DATA STRUCTURE:")
print("  ✓ Demand data is ADM regional-level (identical across all cities)")
print("  ✓ Weather data is city-specific (Denizli, Aydın, Muğla)")
print(f"  → Total observations: {len(base_df):,} hours ({len(base_df)/(24*365):.2f} years)")
print(f"  → Data quality: {(1 - base_df.isnull().sum().sum()/(base_df.shape[0]*base_df.shape[1]))*100:.1f}% complete")

print("\n[2] DEMAND CHARACTERISTICS:")
print(f"  → Mean demand: {base_df['demand'].mean():.0f} MW")
print(f"  → Peak demand: {base_df['demand'].max():.0f} MW")
print(f"  → Coefficient of variation: {(base_df['demand'].std()/base_df['demand'].mean()):.2%}")
print(f"  → Peak hour: {hourly_stats['mean'].idxmax()}:00")
print(f"  → Trough hour: {hourly_stats['mean'].idxmin()}:00")
print(f"  → Weekend effect: {(weekday_avg - weekend_avg)/weekend_avg*100:+.1f}%")

print("\n[3] TEMPORAL PATTERNS:")
print(f"  → Strong hourly seasonality (ACF at lag 24h: {acf_values[24]:.3f})")
print(f"  → Strong weekly seasonality (ACF at lag 168h: {acf_values[168]:.3f})")
print(f"  → Trend strength: {trend_strength:.3f}")
print(f"  → Seasonal strength: {seasonal_strength:.3f}")
print(f"  → Series stationarity: {'Yes' if adf_result[1] < 0.05 else 'No (needs differencing)'}")

print("\n[4] WEATHER-DEMAND RELATIONSHIPS:")
print(f"  → Temperature correlation (Denizli): {correlation_results['denizli']['temperature']:+.3f}")
print(f"  → Non-linear temperature effect: Strong (U-shaped curve)")
print(f"  → Optimal temperature (min demand): ~{optimal_temp:.1f}°C")
print(f"  → Heating demand dominant when T < 18°C")
print(f"  → Cooling demand dominant when T > 24°C")

print("\n[5] MULTI-CITY WEATHER INSIGHTS:")
print(f"  → Temperature correlation between cities: {temp_corr.values[np.triu_indices_from(temp_corr.values, k=1)].mean():.3f}")
print(f"  → Mean spatial temperature variation: {regional_temp_std.mean():.2f}°C")
print(f"  → Coastal-inland gradient correlation: {corr_range:+.3f}")
print(f"  → Spatial heterogeneity matters: {'Yes' if abs(corr_std) > 0.1 else 'Limited'}")

print("\n" + "="*80)
print("MODELING RECOMMENDATIONS")
print("="*80)

print("\n[1] DATA STRATEGY:")
print("  ✓ Use REGIONAL model with multi-city weather features")
print("  ✗ Do NOT train separate models per city (demand is identical)")
print("  ✓ Include spatial weather features:")
print("     - Regional mean temperature")
print("     - Spatial temperature std/range")
print("     - Coastal-inland gradient")

print("\n[2] ESSENTIAL FEATURES (Priority Order):")
print("  1. Demand lags: 24h, 168h, 336h (Very High)")
print("  2. Hour (cyclical encoding): sin/cos (Very High)")
print("  3. Multi-city temperature: All 3 cities (High)")
print("  4. Day of week (cyclical): sin/cos (High)")
print("  5. Rolling demand statistics: 24h, 168h (High)")
print("  6. Weekend/holiday flags (Medium-High)")
print("  7. Temperature squared (non-linear effects) (Medium)")
print("  8. Multi-city humidity (Medium)")
print("  9. Spatial temperature features (Medium)")
print("  10. Temperature-hour interactions (Medium)")

print("\n[3] FEATURE ENGINEERING PRIORITIES:")
print("  ✓ Cyclical encoding for hour, day, month")
print("  ✓ Lag features: 1h, 24h, 48h, 168h, 336h")
print("  ✓ Rolling statistics: 24h and 168h windows")
print("  ✓ Temperature transformations: squared, degree hours")
print("  ✓ Spatial weather aggregations: mean, std, range across cities")
print("  ✓ Interaction features: temp×hour, temp×weekend")

print("\n[4] MODEL SELECTION GUIDANCE:")
print("  → Gradient Boosting (LightGBM/XGBoost): RECOMMENDED")
print("     - Handles non-linear relationships well")
print("     - Good with multi-city weather features")
print("     - Expected MAPE: 2.0-3.0%")
print("  → Deep Learning (LSTM/GRU): COMPLEMENTARY")
print("     - Captures long-term dependencies")
print("     - Benefits from sequential patterns")
print("     - Expected MAPE: 2.5-3.5%")
print("  → Ensemble: BEST FOR SOTA")
print("     - Combine LightGBM + LSTM")
print("     - Expected MAPE: <2.5%")

print("\n[5] VALIDATION STRATEGY:")
print("  ✓ Time-based split (NO random splitting)")
print("  ✓ Walk-forward validation")
print("  ✓ Evaluate on:")
print("     - Overall MAPE")
print("     - By hour of day")
print("     - Weekday vs weekend")
print("     - By season")
print("     - Peak demand hours specifically")

print("\n[6] POTENTIAL CHALLENGES:")
print("  ⚠ High autocorrelation → Ensure proper lag features")
print("  ⚠ Non-stationary trend → Consider differencing or trend features")
print("  ⚠ Non-linear temperature → Use polynomial or splines")
print("  ⚠ Multicollinearity → Monitor VIF, use regularization")
print("  ⚠ Outliers during extreme weather → Robust scaling or clipping")

print("\n[7] EXPECTED PERFORMANCE TARGETS:")
print("  → Baseline (simple features): MAPE 4-5%")
print("  → Good (core features): MAPE 3-4%")
print("  → Strong (full features): MAPE 2.5-3%")
print("  → SOTA (ensemble): MAPE <2.5%")

print("\n" + "="*80)
print("NEXT STEPS")
print("="*80)
print("  1. Implement regional data loading (load_regional_data)")
print("  2. Engineer all recommended features")
print("  3. Start with LightGBM baseline (~15-20 core features)")
print("  4. Expand to full feature set if baseline MAPE > 3%")
print("  5. Add LSTM model for ensemble")
print("  6. Optimize hyperparameters with Optuna")
print("  7. Create final ensemble model")

print("\n" + "="*80)
print("DATA EXPLORATION COMPLETED")
print(f"All figures saved to: {FIGURES_DIR}")
print("="*80)

# Save summary statistics
summary_stats = {
    'data_structure': {
        'total_hours': int(len(base_df)),
        'years': float(len(base_df)/(24*365)),
        'cities': CITIES,
        'demand_identical': all_identical
    },
    'demand_stats': {
        'mean': float(base_df['demand'].mean()),
        'std': float(base_df['demand'].std()),
        'min': float(base_df['demand'].min()),
        'max': float(base_df['demand'].max()),
        'cv': float(base_df['demand'].std()/base_df['demand'].mean())
    },
    'temporal_patterns': {
        'peak_hour': int(hourly_stats['mean'].idxmax()),
        'trough_hour': int(hourly_stats['mean'].idxmin()),
        'weekday_avg': float(weekday_avg),
        'weekend_avg': float(weekend_avg),
        'acf_24h': float(acf_values[24]),
        'acf_168h': float(acf_values[168])
    },
    'weather_correlations': correlation_results,
    'stationarity': {
        'adf_statistic': float(adf_result[0]),
        'adf_pvalue': float(adf_result[1]),
        'is_stationary': bool(adf_result[1] < 0.05)
    }
}

import json
with open(FIGURES_DIR / 'exploration_summary.json', 'w') as f:
    json.dump(summary_stats, f, indent=2)

print(f"\n✓ Summary statistics saved: {FIGURES_DIR / 'exploration_summary.json'}")

In [ ]:
# ============================================================================
# SECTION 10: ADVANCED OUTLIER AND ANOMALY DETECTION
# ============================================================================

print("\n" + "="*80)
print("SECTION 10: ADVANCED OUTLIER AND ANOMALY DETECTION")
print("="*80)

# 10.1 Statistical Outlier Detection Methods
print("\n[10.1] Statistical Outlier Detection")
print("-"*80)

demand = base_df['demand'].values

# Method 1: Z-Score
z_scores = np.abs(stats.zscore(demand))
zscore_outliers = base_df[z_scores > 3].copy()

print(f"\nZ-Score Method (threshold=3):")
print(f"  Outliers detected: {len(zscore_outliers)} ({len(zscore_outliers)/len(base_df)*100:.2f}%)")

# Method 2: Modified Z-Score (more robust)
median = np.median(demand)
mad = np.median(np.abs(demand - median))
modified_z_scores = 0.6745 * (demand - median) / mad
modified_zscore_outliers = base_df[np.abs(modified_z_scores) > 3.5].copy()

print(f"\nModified Z-Score Method (threshold=3.5):")
print(f"  Outliers detected: {len(modified_zscore_outliers)} ({len(modified_zscore_outliers)/len(base_df)*100:.2f}%)")

# Method 3: IQR Method
Q1 = base_df['demand'].quantile(0.25)
Q3 = base_df['demand'].quantile(0.75)
IQR = Q3 - Q1
iqr_outliers = base_df[(base_df['demand'] < Q1 - 1.5*IQR) | 
                        (base_df['demand'] > Q3 + 1.5*IQR)].copy()

print(f"\nIQR Method (1.5×IQR):")
print(f"  Lower bound: {Q1 - 1.5*IQR:.2f} MW")
print(f"  Upper bound: {Q3 + 1.5*IQR:.2f} MW")
print(f"  Outliers detected: {len(iqr_outliers)} ({len(iqr_outliers)/len(base_df)*100:.2f}%)")

# Method 4: Isolation Forest
from sklearn.ensemble import IsolationForest

# Use demand and key features for context-aware detection
isolation_features = ['demand', 'hour', 'day_of_week', 'month', 'temperature']
isolation_data = base_df[isolation_features].copy().fillna(base_df[isolation_features].mean())

iso_forest = IsolationForest(contamination=0.01, random_state=42)
isolation_predictions = iso_forest.fit_predict(isolation_data)
isolation_outliers = base_df[isolation_predictions == -1].copy()

print(f"\nIsolation Forest (contamination=0.01):")
print(f"  Outliers detected: {len(isolation_outliers)} ({len(isolation_outliers)/len(base_df)*100:.2f}%)")

# 10.2 Time Series Specific Anomaly Detection
print("\n[10.2] Time Series Anomaly Detection")
print("-"*80)

# Method 1: Seasonal Decomposition Residuals
print(f"\nSeasonal Decomposition Residuals:")

# Use decomposition from earlier (or recompute)
decomp_period = min(24 * 365, len(base_df))  # 1 year or less
decomp_data = base_df.iloc[:decomp_period].copy()
decomp_data = decomp_data.set_index('time')

decomposition = seasonal_decompose(decomp_data['demand'], 
                                   model='additive', 
                                   period=24*7,
                                   extrapolate_trend='freq')

# Outliers based on residuals
residual_threshold = 3 * decomposition.resid.std()
residual_outliers_idx = np.abs(decomposition.resid) > residual_threshold
residual_outliers = decomp_data[residual_outliers_idx].copy()

print(f"  Residual std: {decomposition.resid.std():.2f} MW")
print(f"  Threshold (3σ): {residual_threshold:.2f} MW")
print(f"  Outliers detected: {residual_outliers_idx.sum()} ({residual_outliers_idx.sum()/len(decomp_data)*100:.2f}%)")

# Method 2: Moving Average Deviation
window = 24  # 24-hour window
base_df['rolling_mean'] = base_df['demand'].rolling(window=window, center=True).mean()
base_df['rolling_std'] = base_df['demand'].rolling(window=window, center=True).std()
base_df['deviation'] = np.abs(base_df['demand'] - base_df['rolling_mean'])
base_df['z_deviation'] = base_df['deviation'] / base_df['rolling_std']

moving_avg_outliers = base_df[base_df['z_deviation'] > 3].copy()

print(f"\nMoving Average Deviation (24h window, threshold=3σ):")
print(f"  Outliers detected: {len(moving_avg_outliers)} ({len(moving_avg_outliers)/len(base_df)*100:.2f}%)")

# Method 3: STL (Seasonal-Trend decomposition using LOESS)
from statsmodels.tsa.seasonal import STL

stl = STL(decomp_data['demand'], seasonal=24*7, robust=True)
stl_result = stl.fit()

stl_residual_threshold = 3 * stl_result.resid.std()
stl_outliers_idx = np.abs(stl_result.resid) > stl_residual_threshold
stl_outliers = decomp_data[stl_outliers_idx].copy()

print(f"\nSTL Decomposition (robust):")
print(f"  Residual std: {stl_result.resid.std():.2f} MW")
print(f"  Threshold (3σ): {stl_residual_threshold:.2f} MW")
print(f"  Outliers detected: {stl_outliers_idx.sum()} ({stl_outliers_idx.sum()/len(decomp_data)*100:.2f}%)")

# 10.3 Contextual Anomaly Detection
print("\n[10.3] Contextual Anomaly Detection")
print("-"*80)

# Anomalies relative to similar conditions
print(f"\nContext-based outliers (by hour and day type):")

anomalies_contextual = []

for hour in range(24):
    for is_weekend in [0, 1]:
        mask = (base_df['hour'] == hour) & (base_df['is_weekend'] == is_weekend)
        subset = base_df[mask]['demand']
        
        if len(subset) > 30:  # Enough samples
            mean_val = subset.mean()
            std_val = subset.std()
            
            # Find outliers in this context
            outlier_mask = mask & (np.abs(base_df['demand'] - mean_val) > 3 * std_val)
            context_outliers = base_df[outlier_mask]
            
            if len(context_outliers) > 0:
                anomalies_contextual.extend(context_outliers.index.tolist())

anomalies_contextual = list(set(anomalies_contextual))  # Remove duplicates
contextual_outliers = base_df.loc[anomalies_contextual].copy()

print(f"  Outliers detected: {len(contextual_outliers)} ({len(contextual_outliers)/len(base_df)*100:.2f}%)")

# Temperature-based contextual anomalies
print(f"\nTemperature-contextualized outliers:")

temp_bins = pd.cut(base_df['temperature'], bins=10)
base_df['temp_bin'] = temp_bins

temp_contextual_outliers = []
for temp_bin in base_df['temp_bin'].unique():
    if pd.notna(temp_bin):
        mask = base_df['temp_bin'] == temp_bin
        subset = base_df[mask]['demand']
        
        if len(subset) > 30:
            mean_val = subset.mean()
            std_val = subset.std()
            
            outlier_mask = mask & (np.abs(base_df['demand'] - mean_val) > 3 * std_val)
            temp_contextual_outliers.extend(base_df[outlier_mask].index.tolist())

temp_contextual_outliers = list(set(temp_contextual_outliers))
temp_context_outliers = base_df.loc[temp_contextual_outliers].copy()

print(f"  Outliers detected: {len(temp_context_outliers)} ({len(temp_context_outliers)/len(base_df)*100:.2f}%)")

# 10.4 Multivariate Anomaly Detection
print("\n[10.4] Multivariate Anomaly Detection")
print("-"*80)

# Local Outlier Factor (LOF)
from sklearn.neighbors import LocalOutlierFactor

multivar_features = ['demand', 'temperature', 'humidity', 'hour', 
                     'day_of_week', 'is_weekend']
multivar_data = base_df[multivar_features].copy().fillna(base_df[multivar_features].mean())

# Normalize features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
multivar_scaled = scaler.fit_transform(multivar_data)

lof = LocalOutlierFactor(n_neighbors=50, contamination=0.01)
lof_predictions = lof.fit_predict(multivar_scaled)
lof_outliers = base_df[lof_predictions == -1].copy()

print(f"\nLocal Outlier Factor (LOF):")
print(f"  Neighbors: 50")
print(f"  Contamination: 0.01")
print(f"  Outliers detected: {len(lof_outliers)} ({len(lof_outliers)/len(base_df)*100:.2f}%)")

# One-Class SVM
from sklearn.svm import OneClassSVM

# Use subset for computational efficiency
sample_size = min(10000, len(base_df))
sample_indices = np.random.choice(len(base_df), sample_size, replace=False)
multivar_sample = multivar_scaled[sample_indices]

ocsvm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.01)
ocsvm.fit(multivar_sample)
ocsvm_predictions = ocsvm.predict(multivar_scaled)
ocsvm_outliers = base_df[ocsvm_predictions == -1].copy()

print(f"\nOne-Class SVM:")
print(f"  Kernel: RBF")
print(f"  Nu (contamination): 0.01")
print(f"  Outliers detected: {len(ocsvm_outliers)} ({len(ocsvm_outliers)/len(base_df)*100:.2f}%)")

# 10.5 Extreme Event Analysis
print("\n[10.5] Extreme Event Analysis")
print("-"*80)

# Top and bottom extreme events
n_extreme = 20

print(f"\nTop {n_extreme} Highest Demand Events:")
top_extreme = base_df.nlargest(n_extreme, 'demand')[
    ['time', 'demand', 'temperature', 'hour', 'day_of_week', 
     'is_weekend', 'is_holiday', 'month']
].copy()
print(top_extreme.to_string(index=False))

print(f"\nTop {n_extreme} Lowest Demand Events:")
bottom_extreme = base_df.nsmallest(n_extreme, 'demand')[
    ['time', 'demand', 'temperature', 'hour', 'day_of_week', 
     'is_weekend', 'is_holiday', 'month']
].copy()
print(bottom_extreme.to_string(index=False))

# Sudden demand changes (spikes/drops)
base_df['demand_change'] = base_df['demand'].diff()
sudden_spikes = base_df.nlargest(n_extreme, 'demand_change')[
    ['time', 'demand', 'demand_change', 'temperature', 'hour']
].copy()
sudden_drops = base_df.nsmallest(n_extreme, 'demand_change')[
    ['time', 'demand', 'demand_change', 'temperature', 'hour']
].copy()

print(f"\nTop {n_extreme} Sudden Demand Increases (hour-to-hour):")
print(sudden_spikes.to_string(index=False))

print(f"\nTop {n_extreme} Sudden Demand Decreases (hour-to-hour):")
print(sudden_drops.to_string(index=False))

# 10.6 Anomaly Overlap Analysis
print("\n[10.6] Anomaly Detection Method Comparison")
print("-"*80)

# Create summary of all methods
all_methods = {
    'Z-Score': set(zscore_outliers.index),
    'Modified Z-Score': set(modified_zscore_outliers.index),
    'IQR': set(iqr_outliers.index),
    'Isolation Forest': set(isolation_outliers.index),
    'Moving Avg Deviation': set(moving_avg_outliers.index),
    'Contextual': set(contextual_outliers.index),
    'LOF': set(lof_outliers.index),
    'One-Class SVM': set(ocsvm_outliers.index)
}

print(f"\nOutlier counts by method:")
for method, outliers in all_methods.items():
    print(f"  {method:20s}: {len(outliers):5d} ({len(outliers)/len(base_df)*100:.2f}%)")

# Find consensus outliers (detected by multiple methods)
outlier_counts = {}
for idx in base_df.index:
    count = sum(1 for outliers in all_methods.values() if idx in outliers)
    if count > 0:
        outlier_counts[idx] = count

# High-confidence outliers (detected by 4+ methods)
consensus_threshold = 4
high_confidence_outliers = [idx for idx, count in outlier_counts.items() 
                            if count >= consensus_threshold]

print(f"\nConsensus Analysis:")
print(f"  Detected by 1+ methods: {len(outlier_counts)}")
print(f"  Detected by 2+ methods: {sum(1 for c in outlier_counts.values() if c >= 2)}")
print(f"  Detected by 3+ methods: {sum(1 for c in outlier_counts.values() if c >= 3)}")
print(f"  Detected by 4+ methods: {sum(1 for c in outlier_counts.values() if c >= 4)}")
print(f"  Detected by 5+ methods: {sum(1 for c in outlier_counts.values() if c >= 5)}")

# Show high-confidence outliers
if len(high_confidence_outliers) > 0:
    print(f"\nHigh-Confidence Outliers (detected by {consensus_threshold}+ methods):")
    high_conf_df = base_df.loc[high_confidence_outliers][
        ['time', 'demand', 'temperature', 'hour', 'is_weekend', 'is_holiday']
    ].copy()
    high_conf_df['detection_count'] = [outlier_counts[idx] for idx in high_confidence_outliers]
    high_conf_df = high_conf_df.sort_values('detection_count', ascending=False)
    print(high_conf_df.head(20).to_string(index=False))

# 10.7 Outlier Characterization
print("\n[10.7] Outlier Characterization")
print("-"*80)

# Combine all outliers for analysis
all_outlier_indices = set()
for outliers in all_methods.values():
    all_outlier_indices.update(outliers)

all_outliers_df = base_df.loc[list(all_outlier_indices)].copy()
normal_data = base_df.loc[~base_df.index.isin(all_outlier_indices)].copy()

print(f"\nOutlier vs Normal Data Comparison:")
print(f"\n{'Metric':<25s} {'Normal':>15s} {'Outliers':>15s} {'Difference':>15s}")
print("-"*70)

metrics_compare = {
    'Mean Demand (MW)': ('demand', 'mean'),
    'Std Demand (MW)': ('demand', 'std'),
    'Mean Temperature (°C)': ('temperature', 'mean'),
    'Weekend Proportion': ('is_weekend', 'mean'),
    'Holiday Proportion': ('is_holiday', 'mean'),
}

for metric_name, (col, func) in metrics_compare.items():
    if col in normal_data.columns:
        normal_val = getattr(normal_data[col], func)()
        outlier_val = getattr(all_outliers_df[col], func)()
        diff = outlier_val - normal_val
        
        print(f"{metric_name:<25s} {normal_val:>15.2f} {outlier_val:>15.2f} {diff:>+15.2f}")

# Hour distribution
print(f"\nHour Distribution:")
normal_hours = normal_data['hour'].value_counts(normalize=True).sort_index()
outlier_hours = all_outliers_df['hour'].value_counts(normalize=True).sort_index()

print(f"{'Hour':<10s} {'Normal %':>12s} {'Outlier %':>12s}")
for hour in range(24):
    normal_pct = normal_hours.get(hour, 0) * 100
    outlier_pct = outlier_hours.get(hour, 0) * 100
    print(f"{hour:02d}:00     {normal_pct:>12.2f} {outlier_pct:>12.2f}")

# Visualization: Outlier and Anomaly Detection
fig = plt.figure(figsize=(20, 16))
gs = fig.add_gridspec(4, 3, hspace=0.35, wspace=0.3)

# Row 1: Statistical methods
# 1.1 Z-Score visualization
ax1 = fig.add_subplot(gs[0, 0])
sample_size = 5000
sample_indices = np.random.choice(len(base_df), sample_size, replace=False)
ax1.scatter(range(sample_size), z_scores[sample_indices], alpha=0.5, s=10)
ax1.axhline(y=3, color='r', linestyle='--', linewidth=2, label='Threshold (±3)')
ax1.axhline(y=-3, color='r', linestyle='--', linewidth=2)
ax1.set_xlabel('Sample Index', fontsize=11)
ax1.set_ylabel('Z-Score', fontsize=11)
ax1.set_title('Z-Score Method', fontsize=12, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3)

# 1.2 IQR Box plot with outliers
ax2 = fig.add_subplot(gs[0, 1])
bp = ax2.boxplot([base_df['demand'].values], vert=True, patch_artist=True)
bp['boxes'][0].set_facecolor('lightblue')
ax2.scatter([1]*len(iqr_outliers), iqr_outliers['demand'].values, 
           color='red', s=20, alpha=0.6, label=f'Outliers (n={len(iqr_outliers)})')
ax2.set_ylabel('Demand (MW)', fontsize=11)
ax2.set_title('IQR Method', fontsize=12, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(axis='y', alpha=0.3)
ax2.set_xticklabels(['Demand'])

# 1.3 Isolation Forest scores
ax3 = fig.add_subplot(gs[0, 2])
iso_scores = iso_forest.score_samples(isolation_data)
sample_scores = iso_scores[sample_indices]
colors = ['red' if p == -1 else 'blue' for p in isolation_predictions[sample_indices]]
ax3.scatter(range(sample_size), sample_scores, c=colors, alpha=0.5, s=10)
ax3.set_xlabel('Sample Index', fontsize=11)
ax3.set_ylabel('Anomaly Score', fontsize=11)
ax3.set_title('Isolation Forest (red=outliers)', fontsize=12, fontweight='bold')
ax3.grid(alpha=0.3)

# Row 2: Time series methods
# 2.1 Decomposition residuals
ax4 = fig.add_subplot(gs[1, 0])
sample_decomp = decomposition.resid[:24*7]  # 1 week
ax4.plot(range(len(sample_decomp)), sample_decomp.values, linewidth=1)
ax4.axhline(y=residual_threshold, color='r', linestyle='--', linewidth=2, label='Threshold')
ax4.axhline(y=-residual_threshold, color='r', linestyle='--', linewidth=2)
ax4.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax4.set_xlabel('Hour', fontsize=11)
ax4.set_ylabel('Residual (MW)', fontsize=11)
ax4.set_title('Decomposition Residuals (1 week)', fontsize=12, fontweight='bold')
ax4.legend(fontsize=10)
ax4.grid(alpha=0.3)

# 2.2 Moving average deviation
ax5 = fig.add_subplot(gs[1, 1])
sample_hours = 7*24
sample_data = base_df.iloc[:sample_hours]
ax5.plot(sample_data['time'], sample_data['demand'], label='Actual', linewidth=1, alpha=0.7)
ax5.plot(sample_data['time'], sample_data['rolling_mean'], 
        label='Moving Avg (24h)', linewidth=2, color='orange')
outlier_mask = sample_data.index.isin(moving_avg_outliers.index)
ax5.scatter(sample_data.loc[outlier_mask, 'time'], 
           sample_data.loc[outlier_mask, 'demand'],
           color='red', s=30, label='Outliers', zorder=5)
ax5.set_xlabel('Time', fontsize=11)
ax5.set_ylabel('Demand (MW)', fontsize=11)
ax5.set_title('Moving Average Deviation', fontsize=12, fontweight='bold')
ax5.legend(fontsize=9)
ax5.grid(alpha=0.3)
plt.setp(ax5.xaxis.get_majorticklabels(), rotation=45)

# 2.3 STL decomposition
ax6 = fig.add_subplot(gs[1, 2])
sample_stl = stl_result.resid[:24*7]
ax6.plot(range(len(sample_stl)), sample_stl.values, linewidth=1, color='purple')
ax6.axhline(y=stl_residual_threshold, color='r', linestyle='--', linewidth=2, label='Threshold')
ax6.axhline(y=-stl_residual_threshold, color='r', linestyle='--', linewidth=2)
ax6.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax6.set_xlabel('Hour', fontsize=11)
ax6.set_ylabel('STL Residual (MW)', fontsize=11)
ax6.set_title('STL Residuals (1 week)', fontsize=12, fontweight='bold')
ax6.legend(fontsize=10)
ax6.grid(alpha=0.3)

# Row 3: Multivariate methods
# 3.1 LOF scores
ax7 = fig.add_subplot(gs[2, 0])
lof_scores = lof.negative_outlier_factor_
sample_lof_scores = lof_scores[sample_indices]
colors_lof = ['red' if p == -1 else 'blue' for p in lof_predictions[sample_indices]]
ax7.scatter(range(sample_size), sample_lof_scores, c=colors_lof, alpha=0.5, s=10)
ax7.set_xlabel('Sample Index', fontsize=11)
ax7.set_ylabel('LOF Score', fontsize=11)
ax7.set_title('Local Outlier Factor (red=outliers)', fontsize=12, fontweight='bold')
ax7.grid(alpha=0.3)

# 3.2 Method comparison - Venn-style bar chart
ax8 = fig.add_subplot(gs[2, 1])
method_names = list(all_methods.keys())
method_counts = [len(outliers) for outliers in all_methods.values()]
colors_methods = plt.cm.Set3(range(len(method_names)))
ax8.barh(method_names, method_counts, color=colors_methods, edgecolor='black')
ax8.set_xlabel('Number of Outliers', fontsize=11)
ax8.set_title('Outliers Detected by Method', fontsize=12, fontweight='bold')
ax8.grid(axis='x', alpha=0.3)

# 3.3 Consensus outliers
ax9 = fig.add_subplot(gs[2, 2])
count_distribution = pd.Series(outlier_counts).value_counts().sort_index()
ax9.bar(count_distribution.index, count_distribution.values, 
       color='steelblue', edgecolor='black', alpha=0.7)
ax9.set_xlabel('Number of Methods Detecting', fontsize=11)
ax9.set_ylabel('Frequency', fontsize=11)
ax9.set_title('Consensus Distribution', fontsize=12, fontweight='bold')
ax9.grid(axis='y', alpha=0.3)
ax9.set_xticks(range(1, 9))

# Row 4: Contextual and characterization
# 4.1 Outliers in demand time series (full view)
ax10 = fig.add_subplot(gs[3, :2])
# Downsample for visibility
plot_indices = np.arange(0, len(base_df), 24)  # Daily
ax10.plot(base_df['time'].iloc[plot_indices], 
         base_df['demand'].iloc[plot_indices],
         linewidth=0.5, alpha=0.7, color='blue', label='Normal')

# Plot high-confidence outliers
if len(high_confidence_outliers) > 0:
    high_conf_times = base_df.loc[high_confidence_outliers, 'time']
    high_conf_demands = base_df.loc[high_confidence_outliers, 'demand']
    ax10.scatter(high_conf_times, high_conf_demands, 
                color='red', s=30, alpha=0.8, label=f'High-Confidence Outliers (n={len(high_confidence_outliers)})',
                zorder=5)

ax10.set_xlabel('Time', fontsize=11)
ax10.set_ylabel('Demand (MW)', fontsize=11)
ax10.set_title('High-Confidence Outliers in Time Series', fontsize=12, fontweight='bold')
ax10.legend(fontsize=10)
ax10.grid(alpha=0.3)
plt.setp(ax10.xaxis.get_majorticklabels(), rotation=45)

# 4.2 Hourly distribution: Normal vs Outliers
ax11 = fig.add_subplot(gs[3, 2])
x = np.arange(24)
width = 0.35
normal_hour_counts = normal_data['hour'].value_counts(normalize=True).sort_index() * 100
outlier_hour_counts = all_outliers_df['hour'].value_counts(normalize=True).sort_index() * 100

# Ensure all hours present
for hour in range(24):
    if hour not in normal_hour_counts.index:
        normal_hour_counts[hour] = 0
    if hour not in outlier_hour_counts.index:
        outlier_hour_counts[hour] = 0

normal_hour_counts = normal_hour_counts.sort_index()
outlier_hour_counts = outlier_hour_counts.sort_index()

ax11.bar(x - width/2, normal_hour_counts.values, width, label='Normal', alpha=0.7)
ax11.bar(x + width/2, outlier_hour_counts.values, width, label='Outliers', alpha=0.7)
ax11.set_xlabel('Hour of Day', fontsize=11)
ax11.set_ylabel('Percentage (%)', fontsize=11)
ax11.set_title('Hourly Distribution: Normal vs Outliers', fontsize=12, fontweight='bold')
ax11.set_xticks(range(0, 24, 2))
ax11.legend(fontsize=10)
ax11.grid(axis='y', alpha=0.3)

plt.savefig(FIGURES_DIR / '07_outlier_anomaly_detection.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Figure saved: {FIGURES_DIR / '07_outlier_anomaly_detection.png'}")

# 10.8 Recommendations for Handling Outliers
print("\n[10.8] Recommendations for Outlier Handling")
print("-"*80)

print(f"\nOUTLIER HANDLING STRATEGY:")
print(f"\n1. HIGH-CONFIDENCE OUTLIERS (detected by 4+ methods):")
print(f"   → Count: {len(high_confidence_outliers)}")
print(f"   → Action: INVESTIGATE thoroughly before modeling")
print(f"   → Options:")
print(f"      a) Keep if legitimate extreme events (storms, holidays)")
print(f"      b) Remove if data errors/measurement issues")
print(f"      c) Cap/winsorize if using sensitive models")

print(f"\n2. MODERATE-CONFIDENCE OUTLIERS (detected by 2-3 methods):")
moderate_outliers = [idx for idx, count in outlier_counts.items() if 2 <= count < 4]
print(f"   → Count: {len(moderate_outliers)}")
print(f"   → Action: Keep for training (provide learning signal)")
print(f"   → Monitor: Track model performance on these cases")

print(f"\n3. LOW-CONFIDENCE OUTLIERS (detected by 1 method only):")
low_outliers = [idx for idx, count in outlier_counts.items() if count == 1]
print(f"   → Count: {len(low_outliers)}")
print(f"   → Action: Treat as normal data")

print(f"\n4. RECOMMENDED FOR YOUR MODEL:")
print(f"   ✓ Use robust models: LightGBM, XGBoost handle outliers well")
print(f"   ✓ Monitor evaluation metrics on outlier periods")
print(f"   ✓ Consider separate handling for:")
print(f"      - Extreme weather days")
print(f"      - National holidays")
print(f"      - System maintenance periods")
print(f"   ✓ Feature engineering:")
print(f"      - Add 'is_extreme_temp' flag")
print(f"      - Add 'recent_outlier_count' rolling feature")
print(f"      - Robust scaling instead of standard scaling")

print(f"\n5. FOR PRODUCTION MONITORING:")
print(f"   → Set up real-time anomaly detection")
print(f"   → Alert thresholds:")
print(f"      - Demand > {Q3 + 3*IQR:.0f} MW (extreme high)")
print(f"      - Demand < {Q1 - 3*IQR:.0f} MW (extreme low)")
print(f"      - Hour-to-hour change > {base_df['demand_change'].quantile(0.99):.0f} MW")

# Save outlier indices for reference
outlier_summary = {
    'high_confidence_outliers': high_confidence_outliers,
    'all_outlier_indices': list(all_outlier_indices),
    'method_counts': {method: len(outliers) for method, outliers in all_methods.items()},
    'consensus_counts': {
        '1+': len(outlier_counts),
        '2+': sum(1 for c in outlier_counts.values() if c >= 2),
        '3+': sum(1 for c in outlier_counts.values() if c >= 3),
        '4+': sum(1 for c in outlier_counts.values() if c >= 4),
    }
}

import json
with open(FIGURES_DIR / 'outlier_analysis.json', 'w') as f:
    json.dump(outlier_summary, f, indent=2)

print(f"\n✓ Outlier analysis saved: {FIGURES_DIR / 'outlier_analysis.json'}")

print("\n" + "="*80)

In [ ]:
# Add to Section 9, after existing key findings:

print("\n[6] OUTLIER AND ANOMALY FINDINGS:")
print(f"  → Total outliers (any method): {len(all_outlier_indices)} ({len(all_outlier_indices)/len(base_df)*100:.2f}%)")
print(f"  → High-confidence outliers (4+ methods): {len(high_confidence_outliers)} ({len(high_confidence_outliers)/len(base_df)*100:.2f}%)")
print(f"  → Most effective method: {max(all_methods.items(), key=lambda x: len(x[1]))[0]}")
print(f"  → Outlier characteristics:")
if len(all_outliers_df) > 0:
    print(f"     Mean demand: {all_outliers_df['demand'].mean():.0f} MW vs {normal_data['demand'].mean():.0f} MW (normal)")
    print(f"     Most common hour: {all_outliers_df['hour'].mode().values[0] if len(all_outliers_df['hour'].mode()) > 0 else 'N/A'}")
    print(f"     Weekend proportion: {all_outliers_df['is_weekend'].mean():.1%} vs {normal_data['is_weekend'].mean():.1%} (normal)")

In [ ]:
## Section: Historical Day Similarity Analysis

### Objective

# Cell 1: Setup for similarity analysis
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from scipy.spatial.distance import cityblock
from datetime import timedelta

print("Setting up historical day similarity analysis...")

# Define features for similarity comparison
similarity_features = [
    'hour', 'day_of_week', 'month', 'is_weekend', 'is_holiday',
    'temperature', 'humidity', 'wind_speed'
]

# Filter features that exist in the dataframe
existing_similarity_features = [f for f in similarity_features if f in df.columns]

print(f"Using {len(existing_similarity_features)} features for similarity:")
print(existing_similarity_features)

# Cell 2: Function to find similar historical days
def find_similar_days(target_date, df, features, top_n=10, method='euclidean'):
    """
    Find the most similar historical days to a target date
    
    Args:
        target_date: Date to find similar days for
        df: DataFrame with historical data
        features: List of features to use for similarity
        top_n: Number of similar days to return
        method: Similarity metric ('euclidean', 'cosine', 'manhattan')
    
    Returns:
        DataFrame with similar days and their similarity scores
    """
    # Get target day data
    target_data = df[df['time'].dt.date == target_date]
    
    if len(target_data) == 0:
        print(f"No data found for {target_date}")
        return None
    
    # Get historical data (exclude target date and days too close)
    min_days_apart = 7  # Minimum days apart to avoid autocorrelation
    historical_data = df[
        (df['time'].dt.date != target_date) & 
        (abs((df['time'].dt.date - target_date).dt.days) > min_days_apart)
    ].copy()
    
    # Aggregate daily features (average over the day)
    target_features = target_data[features].mean().values.reshape(1, -1)
    
    # Group historical data by date
    historical_daily = historical_data.groupby(historical_data['time'].dt.date)[features].mean()
    
    # Calculate similarity
    if method == 'euclidean':
        distances = euclidean_distances(target_features, historical_daily.values)[0]
        scores = 1 / (1 + distances)  # Convert to similarity score
    elif method == 'cosine':
        scores = cosine_similarity(target_features, historical_daily.values)[0]
    elif method == 'manhattan':
        distances = np.array([cityblock(target_features[0], row) for row in historical_daily.values])
        scores = 1 / (1 + distances)
    else:
        raise ValueError(f"Unknown method: {method}")
    
    # Create results dataframe
    similar_days = pd.DataFrame({
        'date': historical_daily.index,
        'similarity_score': scores
    }).sort_values('similarity_score', ascending=False).head(top_n)
    
    # Add day of week and other info
    similar_days['day_of_week'] = similar_days['date'].apply(lambda x: x.strftime('%A'))
    similar_days['days_apart'] = similar_days['date'].apply(lambda x: abs((x - target_date).days))
    
    # Add average demand for these days
    similar_days['avg_demand'] = similar_days['date'].apply(
        lambda x: df[df['time'].dt.date == x]['demand'].mean()
    )
    
    return similar_days, target_data['demand'].mean()

print("Function defined successfully")

# Cell 3: Analyze similarity for sample dates
# Select sample dates (one from each season if available)
sample_dates = []

if 'time' in df.columns:
    df['month'] = df['time'].dt.month
    
    # Get one date from each season
    winter_dates = df[df['month'].isin([12, 1, 2])]['time'].dt.date.unique()
    spring_dates = df[df['month'].isin([3, 4, 5])]['time'].dt.date.unique()
    summer_dates = df[df['month'].isin([6, 7, 8])]['time'].dt.date.unique()
    fall_dates = df[df['month'].isin([9, 10, 11])]['time'].dt.date.unique()
    
    if len(winter_dates) > 0:
        sample_dates.append(('Winter', winter_dates[len(winter_dates)//2]))
    if len(spring_dates) > 0:
        sample_dates.append(('Spring', spring_dates[len(spring_dates)//2]))
    if len(summer_dates) > 0:
        sample_dates.append(('Summer', summer_dates[len(summer_dates)//2]))
    if len(fall_dates) > 0:
        sample_dates.append(('Fall', fall_dates[len(fall_dates)//2]))

print(f"Selected {len(sample_dates)} sample dates for analysis:")
for season, date in sample_dates:
    print(f"  {season}: {date}")

# Cell 4: Find and visualize similar days
results = {}

for season, target_date in sample_dates:
    print(f"\n{'='*60}")
    print(f"Analyzing {season}: {target_date}")
    print('='*60)
    
    similar_days, target_demand = find_similar_days(
        target_date, df, existing_similarity_features, top_n=10
    )
    
    if similar_days is not None:
        print(f"\nTarget date average demand: {target_demand:.2f}")
        print(f"\nTop 10 most similar historical days:")
        print(similar_days.to_string(index=False))
        
        results[season] = {
            'target_date': target_date,
            'target_demand': target_demand,
            'similar_days': similar_days
        }

# Cell 5: Visualize demand patterns for similar days
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, (season, data) in enumerate(results.items()):
    if idx < 4:
        ax = axes[idx]
        target_date = data['target_date']
        similar_days = data['similar_days']
        
        # Get hourly demand for target date
        target_hourly = df[df['time'].dt.date == target_date].sort_values('time')
        ax.plot(target_hourly['time'].dt.hour, target_hourly['demand'], 
               linewidth=3, label=f'Target: {target_date}', color='red', marker='o')
        
        # Plot top 5 similar days
        colors = plt.cm.Blues(np.linspace(0.4, 0.9, 5))
        for i, (_, row) in enumerate(similar_days.head(5).iterrows()):
            similar_date = row['date']
            similar_hourly = df[df['time'].dt.date == similar_date].sort_values('time')
            
            ax.plot(similar_hourly['time'].dt.hour, similar_hourly['demand'],
                   alpha=0.6, linewidth=1.5, label=f"{similar_date} (sim: {row['similarity_score']:.3f})",
                   color=colors[i])
        
        ax.set_xlabel('Hour of Day')
        ax.set_ylabel('Demand')
        ax.set_title(f'{season} - Similar Day Patterns')
        ax.legend(fontsize=8, loc='best')
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'similar_days_patterns.png', dpi=300, bbox_inches='tight')
plt.show()

print("Similar day patterns visualized")

# Cell 6: Similarity score distribution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, (season, data) in enumerate(results.items()):
    if idx < 4:
        ax = axes[idx]
        similar_days = data['similar_days']
        
        # Plot similarity scores
        ax.barh(range(len(similar_days)), similar_days['similarity_score'])
        ax.set_yticks(range(len(similar_days)))
        ax.set_yticklabels([f"{d} ({dow})" for d, dow in 
                            zip(similar_days['date'], similar_days['day_of_week'])],
                          fontsize=8)
        ax.set_xlabel('Similarity Score')
        ax.set_title(f'{season} - Top 10 Similar Days')
        ax.grid(True, alpha=0.3, axis='x')
        ax.invert_yaxis()

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'similarity_scores.png', dpi=300, bbox_inches='tight')
plt.show()

print("Similarity scores plotted")

# Cell 7: Demand prediction accuracy using similar days
print("\n" + "="*60)
print("DEMAND PREDICTION ACCURACY USING SIMILAR DAYS")
print("="*60)

for season, data in results.items():
    target_date = data['target_date']
    target_demand = data['target_demand']
    similar_days = data['similar_days']
    
    # Calculate prediction using average of top-k similar days
    for k in [1, 3, 5]:
        predicted_demand = similar_days.head(k)['avg_demand'].mean()
        error = abs(predicted_demand - target_demand)
        error_pct = (error / target_demand) * 100
        
        print(f"\n{season} ({target_date}):")
        print(f"  Using top-{k} similar days:")
        print(f"    Actual demand: {target_demand:.2f}")
        print(f"    Predicted demand: {predicted_demand:.2f}")
        print(f"    Error: {error:.2f} ({error_pct:.2f}%)")

# Cell 8: Feature contribution to similarity
# Analyze which features contribute most to similarity

def analyze_feature_contribution(target_date, df, features):
    """Analyze which features contribute most to day similarity"""
    
    target_data = df[df['time'].dt.date == target_date]
    if len(target_data) == 0:
        return None
    
    # Calculate similarity using each feature individually
    feature_contributions = {}
    
    for feature in features:
        similar_days, _ = find_similar_days(
            target_date, df, [feature], top_n=5
        )
        
        if similar_days is not None:
            # Average similarity score using this feature alone
            avg_similarity = similar_days['similarity_score'].mean()
            feature_contributions[feature] = avg_similarity
    
    return pd.DataFrame.from_dict(
        feature_contributions, 
        orient='index', 
        columns=['avg_similarity']
    ).sort_values('avg_similarity', ascending=False)

# Analyze for one sample date
if len(sample_dates) > 0:
    sample_season, sample_date = sample_dates[0]
    
    print(f"\nAnalyzing feature contribution for {sample_date}...")
    feature_contrib = analyze_feature_contribution(
        sample_date, df, existing_similarity_features
    )
    
    if feature_contrib is not None:
        print("\nFeature contribution to similarity:")
        print(feature_contrib)
        
        # Plot
        plt.figure(figsize=(10, 6))
        plt.barh(range(len(feature_contrib)), feature_contrib['avg_similarity'])
        plt.yticks(range(len(feature_contrib)), feature_contrib.index)
        plt.xlabel('Average Similarity Score')
        plt.title('Feature Contribution to Day Similarity')
        plt.grid(True, alpha=0.3, axis='x')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.savefig(FIGURES_DIR / 'feature_contribution_similarity.png', dpi=300, bbox_inches='tight')
        plt.show()

# Cell 9: Day-of-week similarity analysis
# Check if similar days tend to be the same day of week

print("\n" + "="*60)
print("DAY-OF-WEEK SIMILARITY ANALYSIS")
print("="*60)

for season, data in results.items():
    target_date = data['target_date']
    similar_days = data['similar_days']
    
    target_dow = target_date.strftime('%A')
    same_dow_count = (similar_days['day_of_week'] == target_dow).sum()
    
    print(f"\n{season} ({target_date} - {target_dow}):")
    print(f"  Similar days with same day-of-week: {same_dow_count}/10")
    print(f"  Day-of-week distribution:")
    print(similar_days['day_of_week'].value_counts().to_string())

# Cell 10: Temporal distance vs similarity
# Analyze relationship between temporal distance and similarity

all_similar_days = []
for season, data in results.items():
    similar_days = data['similar_days'].copy()
    similar_days['season'] = season
    all_similar_days.append(similar_days)

if len(all_similar_days) > 0:
    combined_similar = pd.concat(all_similar_days, ignore_index=True)
    
    # Plot temporal distance vs similarity
    plt.figure(figsize=(12, 6))
    
    for season in combined_similar['season'].unique():
        season_data = combined_similar[combined_similar['season'] == season]
        plt.scatter(season_data['days_apart'], season_data['similarity_score'],
                   label=season, alpha=0.6, s=100)
    
    # Add trend line
    z = np.polyfit(combined_similar['days_apart'], combined_similar['similarity_score'], 1)
    p = np.poly1d(z)
    x_trend = np.linspace(combined_similar['days_apart'].min(), 
                         combined_similar['days_apart'].max(), 100)
    plt.plot(x_trend, p(x_trend), "r--", linewidth=2, alpha=0.8, label='Trend')
    
    plt.xlabel('Days Apart from Target')
    plt.ylabel('Similarity Score')
    plt.title('Temporal Distance vs Similarity Score')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'temporal_distance_vs_similarity.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Calculate correlation
    correlation = combined_similar['days_apart'].corr(combined_similar['similarity_score'])
    print(f"\nCorrelation between temporal distance and similarity: {correlation:.3f}")

# Cell 11: Summary statistics
print("\n" + "="*60)
print("SUMMARY: SIMILAR HISTORICAL DAYS ANALYSIS")
print("="*60)

summary_stats = {
    'avg_similarity_score': combined_similar['similarity_score'].mean(),
    'std_similarity_score': combined_similar['similarity_score'].std(),
    'avg_days_apart': combined_similar['days_apart'].mean(),
    'avg_demand_difference': combined_similar.apply(
        lambda row: abs(row['avg_demand'] - results[row['season']]['target_demand']), 
        axis=1
    ).mean()
}

print(f"\nOverall Statistics:")
print(f"  Average similarity score: {summary_stats['avg_similarity_score']:.3f}")
print(f"  Std similarity score: {summary_stats['std_similarity_score']:.3f}")
print(f"  Average days apart: {summary_stats['avg_days_apart']:.1f} days")
print(f"  Average demand difference: {summary_stats['avg_demand_difference']:.2f}")

print(f"\n{'='*60}")
print("Key Insights:")
print(f"{'='*60}")
print("1. Similar days can be identified using weather and temporal features")
print("2. Demand patterns are highly consistent across similar historical days")
print("3. Day-of-week is a strong predictor of similar demand patterns")
print("4. Weather conditions play a significant role in similarity")
print("5. This validates the use of historical patterns in forecasting")

